In [1]:
"""
# ════════════════════════════════════════════════════════════════════════════════
# CHRONOS-2 USD/PEN - VERSIÓN CORREGIDA
# ════════════════════════════════════════════════════════════════════════════════
#
# PROBLEMA ORIGINAL: DA = 40% (peor que random walk 50%)
# CAUSA: Sesgo sistemático - Chronos predice mayoritariamente "sube"
#
# CORRECCIONES IMPLEMENTADAS:
# 1. Log-Level: Usar log(PEN) para escala más manejable
# 2. Bias Correction: Ajustar por sesgo histórico estimado
# 3. Detrended: Remover tendencia antes de predecir
#
# NOTA: Si ninguna estrategia funciona (DA < 48%), DESCARTAR el modelo
#
# ════════════════════════════════════════════════════════════════════════════════
"""

# ============================================================================
# CELDA 1: IMPORTS Y CONFIGURACIÓN
# ============================================================================

import numpy as np
import pandas as pd
import torch
import json
import pickle
import logging
import warnings
import os
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Tuple, Optional

# Chronos-2
try:
    from chronos import Chronos2Pipeline
    CHRONOS_AVAILABLE = True
except ImportError:
    print("⚠️ Chronos-2 no instalado. Ejecutar: pip install chronos-forecasting")
    CHRONOS_AVAILABLE = False

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

# Logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger(__name__)

# Seeds
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

# Device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

logger.info("=" * 80)
logger.info("CHRONOS-2 USD/PEN - VERSIÓN CORREGIDA")
logger.info("=" * 80)
logger.info(f"Device: {DEVICE}")
logger.info("=" * 80)


c:\Users\Carlos Palma\miniconda3\envs\chronos2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
16:17:45 - INFO - ================================================================================
16:17:45 - INFO - CHRONOS-2 USD/PEN - VERSIÓN CORREGIDA
16:17:45 - INFO - ================================================================================
16:17:45 - INFO - Device: cuda
16:17:45 - INFO - ================================================================================


In [3]:
# ============================================================================
# CELDA 2: CONFIGURACIÓN Y RUTAS
# ============================================================================

# ═══════════════════════════════════════════════════════════════════════════════
# RUTAS - WINDOWS LOCAL (VS Code)
# ═══════════════════════════════════════════════════════════════════════════════

BASE_DIR = Path(r"C:\Users\Carlos Palma\OneDrive\Documents\Cursos\UTEC Computer Science\TESIS\NUEVO PAPER\tesis_maestria")
DATA_PATH = BASE_DIR / "DATA.csv"

OUTPUT_DIR = BASE_DIR / "Chronos2_h1_USD_PEN"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

CHECKPOINT_DIR = OUTPUT_DIR / 'checkpoints'
CHECKPOINT_DIR.mkdir(exist_ok=True)

OOF_DIR = BASE_DIR / "oof_predictions"
OOF_DIR.mkdir(parents=True, exist_ok=True)

PRED_DUMP_DIR = BASE_DIR / "predictions_dump"
PRED_DUMP_DIR.mkdir(parents=True, exist_ok=True)

FIGURES_DIR = OUTPUT_DIR / 'figures'
FIGURES_DIR.mkdir(exist_ok=True)

TABLES_DIR = OUTPUT_DIR / 'tables'
TABLES_DIR.mkdir(exist_ok=True)

# Parámetros estándar
TARGET_COL = 'PEN'  # Niveles
H_FORECAST = 1
N_HOLDOUT = 60
MIN_TRAIN = 252
STEP_SIZE = 21

# Baselines
BASELINES = {
    'Naive RW': {'DA': 50.00, 'MASE': 1.0000},
    'ARX': {'DA': 51.67, 'MASE': 0.9398},
    'CatBoost': {'DA': 50.00, 'MASE': 0.9067},
}

logger.info(f"Data path: {DATA_PATH}")
logger.info(f"Output dir: {OUTPUT_DIR}")


16:36:14 - INFO - Data path: C:\Users\Carlos Palma\OneDrive\Documents\Cursos\UTEC Computer Science\TESIS\NUEVO PAPER\tesis_maestria\DATA.csv
16:36:14 - INFO - Output dir: C:\Users\Carlos Palma\OneDrive\Documents\Cursos\UTEC Computer Science\TESIS\NUEVO PAPER\tesis_maestria\Chronos2_h1_USD_PEN


In [4]:
# ============================================================================
# CELDA 3: MÉTRICAS
# ============================================================================

class FXMetrics:
    """Métricas para evaluación"""
    
    @staticmethod
    def directional_accuracy_from_levels(y_true_levels, y_pred_levels, y_prev_levels):
        """DA desde niveles"""
        true_returns = np.log(y_true_levels / y_prev_levels)
        pred_returns = np.log(y_pred_levels / y_prev_levels)
        
        true_dir = np.sign(true_returns)
        pred_dir = np.sign(pred_returns)
        
        return (np.sum(true_dir == pred_dir) / len(true_dir)) * 100
    
    @staticmethod
    def mase_from_levels(y_true_levels, y_pred_levels, y_prev_levels, train_levels):
        """MASE desde niveles"""
        true_returns = np.log(y_true_levels / y_prev_levels)
        pred_returns = np.log(y_pred_levels / y_prev_levels)
        
        model_mae = np.mean(np.abs(true_returns - pred_returns))
        
        train_returns = np.log(train_levels[1:] / train_levels[:-1])
        naive_mae = np.mean(np.abs(train_returns))
        
        return model_mae / naive_mae if naive_mae > 0 else np.inf
    
    @staticmethod
    def directional_accuracy(y_true, y_pred):
        """DA directo"""
        true_dir = np.sign(y_true)
        pred_dir = np.sign(y_pred)
        return np.sum(true_dir == pred_dir) / len(y_true) * 100

    @staticmethod
    def mase(y_true, y_pred, y_train):
        """MASE directo"""
        model_mae = np.mean(np.abs(y_true - y_pred))
        naive_mae = np.mean(np.abs(np.diff(y_train)))
        return model_mae / naive_mae if naive_mae > 0 else np.inf

logger.info("✓ FXMetrics definido")

16:36:37 - INFO - ✓ FXMetrics definido


In [6]:
# ============================================================================
# CELDA 4: CARGAR DATOS
# ============================================================================

print("\n" + "="*80)
print("📂 CARGANDO DATOS")
print("="*80)

df = pd.read_csv(DATA_PATH)
df['Dates'] = pd.to_datetime(df['Dates'])
df = df.set_index('Dates').sort_index()

# Crear features necesarias
df['PEN_lag_1'] = df['PEN'].shift(1)
df['PEN_log_ret'] = np.log(df['PEN'] / df['PEN'].shift(1))
df['log_PEN'] = np.log(df['PEN'])  # Para estrategia Log-Level
df['log_PEN_lag_1'] = df['log_PEN'].shift(1)

df = df.dropna()

# Split
train_df = df.iloc[:-N_HOLDOUT].copy()
holdout_df = df.iloc[-N_HOLDOUT:].copy()

logger.info(f"✅ Datos cargados: {len(df)} observaciones")
logger.info(f"   Train: {len(train_df)} ({train_df.index.min()} a {train_df.index.max()})")
logger.info(f"   Holdout: {len(holdout_df)} ({holdout_df.index.min()} a {holdout_df.index.max()})")


17:05:43 - INFO - ✅ Datos cargados: 8200 observaciones
17:05:43 - INFO -    Train: 8140 (1994-02-01 00:00:00 a 2025-04-14 00:00:00)
17:05:43 - INFO -    Holdout: 60 (2025-04-15 00:00:00 a 2025-07-07 00:00:00)



📂 CARGANDO DATOS


In [7]:
# ============================================================================
# CELDA 5: CARGAR MODELO CHRONOS-2
# ============================================================================

print("\n" + "="*80)
print("🤖 CARGANDO CHRONOS-2")
print("="*80)

if CHRONOS_AVAILABLE:
    pipeline = Chronos2Pipeline.from_pretrained(
        "amazon/chronos-2",
        device_map=DEVICE,
        torch_dtype=torch.float32
    )
    logger.info(f"✅ Chronos-2 cargado en {DEVICE}")
else:
    raise ImportError("Chronos-2 no disponible")



🤖 CARGANDO CHRONOS-2


`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!
17:05:54 - INFO - ✅ Chronos-2 cargado en cuda


In [8]:
# ============================================================================
# CELDA 6: ESTRATEGIA 1 - LOG-LEVEL (Baseline)
# ============================================================================

def rolling_forecast_log_level(train_df, holdout_df, pipeline):
    """
    Estrategia 1: Usar log(PEN) en lugar de PEN
    
    Hipótesis: La escala logarítmica es más manejable para el modelo
    """
    print("\n" + "="*60)
    print("📊 ESTRATEGIA 1: LOG-LEVEL")
    print("="*60)
    
    n_holdout = len(holdout_df)
    pred_log_levels = []
    
    full_df = pd.concat([train_df, holdout_df])
    
    for i in range(n_holdout):
        if i % 10 == 0:
            logger.info(f"   Día {i+1}/{n_holdout}...")
        
        context_end = len(train_df) + i
        context_df = full_df.iloc[:context_end].copy()
        
        # Usar LOG del nivel
        input_df = pd.DataFrame({
            'id': 'USDPEN',
            'timestamp': context_df.index,
            'target': context_df['log_PEN'].values  # ← LOG
        })
        
        input_df = input_df.set_index('timestamp').asfreq('B').ffill().reset_index()
        input_df = input_df.rename(columns={'index': 'timestamp'})
        
        try:
            pred_df = pipeline.predict_df(
                input_df, prediction_length=1, quantile_levels=[0.5],
                id_column='id', timestamp_column='timestamp', target='target'
            )
            pred_log_level = float(pred_df['predictions'].iloc[0])
            pred_log_levels.append(pred_log_level)
        except Exception as e:
            logger.warning(f"   ⚠️ Error día {i}: {e}")
            # Fallback: persistencia
            pred_log_levels.append(context_df['log_PEN'].iloc[-1])
    
    # Convertir predicciones log → nivel → retorno
    pred_log_levels = np.array(pred_log_levels)
    pred_levels = np.exp(pred_log_levels)
    
    # Métricas
    y_true_levels = holdout_df['PEN'].values
    y_prev_levels = holdout_df['PEN_lag_1'].values
    
    da = FXMetrics.directional_accuracy_from_levels(
        y_true_levels, pred_levels, y_prev_levels
    )
    mase = FXMetrics.mase_from_levels(
        y_true_levels, pred_levels, y_prev_levels, train_df['PEN'].values
    )
    
    # Diagnóstico de sesgo
    pred_returns = np.log(pred_levels / y_prev_levels)
    true_returns = np.log(y_true_levels / y_prev_levels)
    
    pct_pred_up = (pred_returns > 0).mean() * 100
    pct_true_up = (true_returns > 0).mean() * 100
    
    print(f"\n📊 RESULTADOS LOG-LEVEL:")
    print(f"   DA:   {da:.2f}%")
    print(f"   MASE: {mase:.4f}")
    print(f"\n📊 DIAGNÓSTICO DE SESGO:")
    print(f"   % Pred SUBE: {pct_pred_up:.1f}%")
    print(f"   % Real SUBE: {pct_true_up:.1f}%")
    
    return {
        'strategy': 'log_level',
        'pred_levels': pred_levels,
        'pred_returns': pred_returns,
        'DA': da,
        'MASE': mase,
        'pct_pred_up': pct_pred_up,
        'pct_true_up': pct_true_up
    }


In [9]:
# ============================================================================
# CELDA 7: ESTRATEGIA 2 - DETRENDED (Remover tendencia)
# ============================================================================

def rolling_forecast_detrended(train_df, holdout_df, pipeline, window=21):
    """
    Estrategia 2: Remover tendencia móvil antes de predecir
    
    1. Calcular media móvil de los últimos `window` días
    2. Predecir la desviación respecto a la media
    3. Sumar la media a la predicción
    
    Hipótesis: Chronos tiene sesgo de tendencia, esto lo neutraliza
    """
    print("\n" + "="*60)
    print(f"📊 ESTRATEGIA 2: DETRENDED (window={window})")
    print("="*60)
    
    n_holdout = len(holdout_df)
    pred_levels = []
    
    full_df = pd.concat([train_df, holdout_df])
    
    for i in range(n_holdout):
        if i % 10 == 0:
            logger.info(f"   Día {i+1}/{n_holdout}...")
        
        context_end = len(train_df) + i
        context_df = full_df.iloc[:context_end].copy()
        
        # Calcular media móvil
        ma = context_df['PEN'].rolling(window=window).mean()
        last_ma = ma.iloc[-1]
        
        # Crear serie detrended
        detrended = context_df['PEN'] - ma
        detrended = detrended.fillna(0)
        
        # Predecir desviación
        input_df = pd.DataFrame({
            'id': 'USDPEN',
            'timestamp': context_df.index,
            'target': detrended.values
        })
        
        input_df = input_df.set_index('timestamp').asfreq('B').ffill().reset_index()
        input_df = input_df.rename(columns={'index': 'timestamp'})
        
        try:
            pred_df = pipeline.predict_df(
                input_df, prediction_length=1, quantile_levels=[0.5],
                id_column='id', timestamp_column='timestamp', target='target'
            )
            pred_deviation = float(pred_df['predictions'].iloc[0])
            pred_level = last_ma + pred_deviation  # Sumar media
            pred_levels.append(pred_level)
        except Exception as e:
            logger.warning(f"   ⚠️ Error día {i}: {e}")
            pred_levels.append(context_df['PEN'].iloc[-1])
    
    pred_levels = np.array(pred_levels)
    
    # Métricas
    y_true_levels = holdout_df['PEN'].values
    y_prev_levels = holdout_df['PEN_lag_1'].values
    
    da = FXMetrics.directional_accuracy_from_levels(
        y_true_levels, pred_levels, y_prev_levels
    )
    mase = FXMetrics.mase_from_levels(
        y_true_levels, pred_levels, y_prev_levels, train_df['PEN'].values
    )
    
    # Diagnóstico
    pred_returns = np.log(pred_levels / y_prev_levels)
    true_returns = np.log(y_true_levels / y_prev_levels)
    
    pct_pred_up = (pred_returns > 0).mean() * 100
    pct_true_up = (true_returns > 0).mean() * 100
    
    print(f"\n📊 RESULTADOS DETRENDED:")
    print(f"   DA:   {da:.2f}%")
    print(f"   MASE: {mase:.4f}")
    print(f"\n📊 DIAGNÓSTICO DE SESGO:")
    print(f"   % Pred SUBE: {pct_pred_up:.1f}%")
    print(f"   % Real SUBE: {pct_true_up:.1f}%")
    
    return {
        'strategy': f'detrended_w{window}',
        'pred_levels': pred_levels,
        'pred_returns': pred_returns,
        'DA': da,
        'MASE': mase,
        'pct_pred_up': pct_pred_up,
        'pct_true_up': pct_true_up
    }


In [10]:
# ============================================================================
# CELDA 8: ESTRATEGIA 3 - BIAS-CORRECTED
# ============================================================================

def rolling_forecast_bias_corrected(train_df, holdout_df, pipeline, calibration_window=60):
    """
    Estrategia 3: Corrección de sesgo post-hoc
    
    1. Predecir usando niveles normales
    2. Estimar sesgo usando últimas `calibration_window` observaciones
    3. Ajustar predicción restando el sesgo estimado
    
    Hipótesis: El sesgo de Chronos es sistemático y estable
    """
    print("\n" + "="*60)
    print(f"📊 ESTRATEGIA 3: BIAS-CORRECTED (window={calibration_window})")
    print("="*60)
    
    n_holdout = len(holdout_df)
    pred_levels = []
    pred_levels_raw = []  # Sin corrección
    
    full_df = pd.concat([train_df, holdout_df])
    
    # Primero, estimar sesgo en período de calibración (últimos días de train)
    print("   ⏳ Calibrando sesgo...")
    cal_start = len(train_df) - calibration_window
    cal_errors = []
    
    for i in range(calibration_window):
        idx = cal_start + i
        context_df = full_df.iloc[:idx].copy()
        actual_next = full_df.iloc[idx]['PEN']
        prev_level = context_df['PEN'].iloc[-1]
        
        input_df = pd.DataFrame({
            'id': 'USDPEN',
            'timestamp': context_df.index,
            'target': context_df['PEN'].values
        })
        input_df = input_df.set_index('timestamp').asfreq('B').ffill().reset_index()
        input_df = input_df.rename(columns={'index': 'timestamp'})
        
        try:
            pred_df = pipeline.predict_df(
                input_df, prediction_length=1, quantile_levels=[0.5],
                id_column='id', timestamp_column='timestamp', target='target'
            )
            pred_level = float(pred_df['predictions'].iloc[0])
            
            # Error = predicho - real
            pred_change = pred_level - prev_level
            actual_change = actual_next - prev_level
            error = pred_change - actual_change
            cal_errors.append(error)
        except:
            pass
    
    # Sesgo estimado (media del error)
    bias = np.mean(cal_errors) if cal_errors else 0
    print(f"   📊 Sesgo estimado: {bias:.6f}")
    print(f"   📊 Std del error: {np.std(cal_errors):.6f}")
    
    # Ahora predecir en holdout con corrección
    print("   ⏳ Prediciendo holdout...")
    
    for i in range(n_holdout):
        if i % 10 == 0:
            logger.info(f"   Día {i+1}/{n_holdout}...")
        
        context_end = len(train_df) + i
        context_df = full_df.iloc[:context_end].copy()
        prev_level = context_df['PEN'].iloc[-1]
        
        input_df = pd.DataFrame({
            'id': 'USDPEN',
            'timestamp': context_df.index,
            'target': context_df['PEN'].values
        })
        input_df = input_df.set_index('timestamp').asfreq('B').ffill().reset_index()
        input_df = input_df.rename(columns={'index': 'timestamp'})
        
        try:
            pred_df = pipeline.predict_df(
                input_df, prediction_length=1, quantile_levels=[0.5],
                id_column='id', timestamp_column='timestamp', target='target'
            )
            pred_level_raw = float(pred_df['predictions'].iloc[0])
            pred_levels_raw.append(pred_level_raw)
            
            # Corrección de sesgo
            pred_change = pred_level_raw - prev_level
            corrected_change = pred_change - bias
            pred_level = prev_level + corrected_change
            
            pred_levels.append(pred_level)
        except Exception as e:
            logger.warning(f"   ⚠️ Error día {i}: {e}")
            pred_levels.append(context_df['PEN'].iloc[-1])
            pred_levels_raw.append(context_df['PEN'].iloc[-1])
    
    pred_levels = np.array(pred_levels)
    pred_levels_raw = np.array(pred_levels_raw)
    
    # Métricas
    y_true_levels = holdout_df['PEN'].values
    y_prev_levels = holdout_df['PEN_lag_1'].values
    
    # Con corrección
    da = FXMetrics.directional_accuracy_from_levels(
        y_true_levels, pred_levels, y_prev_levels
    )
    mase = FXMetrics.mase_from_levels(
        y_true_levels, pred_levels, y_prev_levels, train_df['PEN'].values
    )
    
    # Sin corrección (para comparar)
    da_raw = FXMetrics.directional_accuracy_from_levels(
        y_true_levels, pred_levels_raw, y_prev_levels
    )
    
    # Diagnóstico
    pred_returns = np.log(pred_levels / y_prev_levels)
    true_returns = np.log(y_true_levels / y_prev_levels)
    
    pct_pred_up = (pred_returns > 0).mean() * 100
    pct_true_up = (true_returns > 0).mean() * 100
    
    print(f"\n📊 RESULTADOS BIAS-CORRECTED:")
    print(f"   DA (corregido):     {da:.2f}%")
    print(f"   DA (sin corregir):  {da_raw:.2f}%")
    print(f"   MASE:               {mase:.4f}")
    print(f"\n📊 DIAGNÓSTICO DE SESGO:")
    print(f"   % Pred SUBE: {pct_pred_up:.1f}%")
    print(f"   % Real SUBE: {pct_true_up:.1f}%")
    
    return {
        'strategy': 'bias_corrected',
        'pred_levels': pred_levels,
        'pred_levels_raw': pred_levels_raw,
        'pred_returns': pred_returns,
        'DA': da,
        'DA_raw': da_raw,
        'MASE': mase,
        'bias': bias,
        'pct_pred_up': pct_pred_up,
        'pct_true_up': pct_true_up
    }


In [11]:
# ============================================================================
# CELDA 9: ESTRATEGIA 4 - LEVEL BASELINE (Sin cambios)
# ============================================================================

def rolling_forecast_level_baseline(train_df, holdout_df, pipeline):
    """
    Baseline: Niveles normales sin modificación
    Para comparar con las estrategias de corrección
    """
    print("\n" + "="*60)
    print("📊 BASELINE: NIVELES SIN MODIFICACIÓN")
    print("="*60)
    
    n_holdout = len(holdout_df)
    pred_levels = []
    
    full_df = pd.concat([train_df, holdout_df])
    
    for i in range(n_holdout):
        if i % 10 == 0:
            logger.info(f"   Día {i+1}/{n_holdout}...")
        
        context_end = len(train_df) + i
        context_df = full_df.iloc[:context_end].copy()
        
        input_df = pd.DataFrame({
            'id': 'USDPEN',
            'timestamp': context_df.index,
            'target': context_df['PEN'].values
        })
        
        input_df = input_df.set_index('timestamp').asfreq('B').ffill().reset_index()
        input_df = input_df.rename(columns={'index': 'timestamp'})
        
        try:
            pred_df = pipeline.predict_df(
                input_df, prediction_length=1, quantile_levels=[0.5],
                id_column='id', timestamp_column='timestamp', target='target'
            )
            pred_level = float(pred_df['predictions'].iloc[0])
            pred_levels.append(pred_level)
        except Exception as e:
            logger.warning(f"   ⚠️ Error día {i}: {e}")
            pred_levels.append(context_df['PEN'].iloc[-1])
    
    pred_levels = np.array(pred_levels)
    
    # Métricas
    y_true_levels = holdout_df['PEN'].values
    y_prev_levels = holdout_df['PEN_lag_1'].values
    
    da = FXMetrics.directional_accuracy_from_levels(
        y_true_levels, pred_levels, y_prev_levels
    )
    mase = FXMetrics.mase_from_levels(
        y_true_levels, pred_levels, y_prev_levels, train_df['PEN'].values
    )
    
    pred_returns = np.log(pred_levels / y_prev_levels)
    true_returns = np.log(y_true_levels / y_prev_levels)
    
    pct_pred_up = (pred_returns > 0).mean() * 100
    pct_true_up = (true_returns > 0).mean() * 100
    
    print(f"\n📊 RESULTADOS BASELINE:")
    print(f"   DA:   {da:.2f}%")
    print(f"   MASE: {mase:.4f}")
    print(f"\n📊 DIAGNÓSTICO DE SESGO:")
    print(f"   % Pred SUBE: {pct_pred_up:.1f}%")
    print(f"   % Real SUBE: {pct_true_up:.1f}%")
    
    return {
        'strategy': 'baseline_levels',
        'pred_levels': pred_levels,
        'pred_returns': pred_returns,
        'DA': da,
        'MASE': mase,
        'pct_pred_up': pct_pred_up,
        'pct_true_up': pct_true_up
    }


In [12]:
# ============================================================================
# CELDA 10: EJECUTAR TODAS LAS ESTRATEGIAS
# ============================================================================

print("\n" + "="*80)
print("🧪 EJECUTANDO ABLATION STUDY DE ESTRATEGIAS")
print("="*80)

all_results = {}

# Baseline
print("\n🔄 Ejecutando BASELINE...")
all_results['baseline'] = rolling_forecast_level_baseline(train_df, holdout_df, pipeline)

# Estrategia 1: Log-Level
print("\n🔄 Ejecutando LOG-LEVEL...")
all_results['log_level'] = rolling_forecast_log_level(train_df, holdout_df, pipeline)

# Estrategia 2: Detrended
print("\n🔄 Ejecutando DETRENDED...")
all_results['detrended'] = rolling_forecast_detrended(train_df, holdout_df, pipeline, window=21)

# Estrategia 3: Bias-Corrected
print("\n🔄 Ejecutando BIAS-CORRECTED...")
all_results['bias_corrected'] = rolling_forecast_bias_corrected(train_df, holdout_df, pipeline)

17:06:59 - INFO -    Día 1/60...



🧪 EJECUTANDO ABLATION STUDY DE ESTRATEGIAS

🔄 Ejecutando BASELINE...

📊 BASELINE: NIVELES SIN MODIFICACIÓN


17:07:01 - INFO -    Día 11/60...
17:07:02 - INFO -    Día 21/60...
17:07:04 - INFO -    Día 31/60...
17:07:05 - INFO -    Día 41/60...
17:07:06 - INFO -    Día 51/60...
17:07:08 - INFO -    Día 1/60...



📊 RESULTADOS BASELINE:
   DA:   40.00%
   MASE: 1.4113

📊 DIAGNÓSTICO DE SESGO:
   % Pred SUBE: 51.7%
   % Real SUBE: 36.7%

🔄 Ejecutando LOG-LEVEL...

📊 ESTRATEGIA 1: LOG-LEVEL


17:07:09 - INFO -    Día 11/60...
17:07:10 - INFO -    Día 21/60...
17:07:12 - INFO -    Día 31/60...
17:07:13 - INFO -    Día 41/60...
17:07:14 - INFO -    Día 51/60...
17:07:16 - INFO -    Día 1/60...


...


📊 RESULTADOS DETRENDED:
   DA:   46.67%
   MASE: 1.4117

📊 DIAGNÓSTICO DE SESGO:
   % Pred SUBE: 75.0%
   % Real SUBE: 36.7%

🔄 Ejecutando BIAS-CORRECTED...

📊 ESTRATEGIA 3: BIAS-CORRECTED (window=60)
   ⏳ Calibrando sesgo...


17:07:32 - INFO -    Día 1/60...


   📊 Sesgo estimado: -0.001120
   📊 Std del error: 0.015756
   ⏳ Prediciendo holdout...


17:07:34 - INFO -    Día 11/60...
17:07:35 - INFO -    Día 21/60...
17:07:36 - INFO -    Día 31/60...
17:07:38 - INFO -    Día 41/60...
17:07:39 - INFO -    Día 51/60...



📊 RESULTADOS BIAS-CORRECTED:
   DA (corregido):     41.67%
   DA (sin corregir):  40.00%
   MASE:               1.4447

📊 DIAGNÓSTICO DE SESGO:
   % Pred SUBE: 60.0%
   % Real SUBE: 36.7%


In [13]:
# ============================================================================
# CELDA 11: COMPARACIÓN Y SELECCIÓN
# ============================================================================

print("\n" + "="*80)
print("📊 COMPARACIÓN DE ESTRATEGIAS")
print("="*80)

print(f"\n{'Estrategia':<20} {'DA':>8} {'MASE':>10} {'% Pred↑':>10} {'% Real↑':>10}")
print("-"*60)

for name, res in all_results.items():
    print(f"{name:<20} {res['DA']:>7.2f}% {res['MASE']:>10.4f} {res['pct_pred_up']:>9.1f}% {res['pct_true_up']:>9.1f}%")

# Encontrar mejor estrategia
best_strategy = max(all_results.keys(), key=lambda k: all_results[k]['DA'])
best_results = all_results[best_strategy]

print(f"\n✅ MEJOR ESTRATEGIA: {best_strategy}")
print(f"   DA:   {best_results['DA']:.2f}%")
print(f"   MASE: {best_results['MASE']:.4f}")


📊 COMPARACIÓN DE ESTRATEGIAS

Estrategia                 DA       MASE    % Pred↑    % Real↑
------------------------------------------------------------
baseline               40.00%     1.4113      51.7%      36.7%
log_level              45.00%     1.4178      55.0%      36.7%
detrended              46.67%     1.4117      75.0%      36.7%
bias_corrected         41.67%     1.4447      60.0%      36.7%

✅ MEJOR ESTRATEGIA: detrended
   DA:   46.67%
   MASE: 1.4117


In [14]:
# ============================================================================
# CELDA 12: DECISIÓN FINAL
# ============================================================================

print("\n" + "="*80)
print("🎯 DECISIÓN FINAL")
print("="*80)

THRESHOLD_DA = 48.0  # Mínimo aceptable (ligeramente bajo random walk)

if best_results['DA'] >= THRESHOLD_DA:
    print(f"\n✅ CHRONOS-2 APROBADO")
    print(f"   Mejor estrategia: {best_strategy}")
    print(f"   DA: {best_results['DA']:.2f}% >= {THRESHOLD_DA}%")
    
    INCLUDE_IN_ENSEMBLE = True
    FINAL_STRATEGY = best_strategy
    FINAL_PREDICTIONS = best_results['pred_levels']
    
else:
    print(f"\n❌ CHRONOS-2 DESCARTADO")
    print(f"   Mejor DA: {best_results['DA']:.2f}% < {THRESHOLD_DA}%")
    print(f"   Ninguna estrategia logra superar el umbral mínimo")
    print(f"   Recomendación: NO incluir en el ensemble")
    
    INCLUDE_IN_ENSEMBLE = False
    FINAL_STRATEGY = None
    FINAL_PREDICTIONS = None



🎯 DECISIÓN FINAL

❌ CHRONOS-2 DESCARTADO
   Mejor DA: 46.67% < 48.0%
   Ninguna estrategia logra superar el umbral mínimo
   Recomendación: NO incluir en el ensemble


In [15]:
# ============================================================================
# CELDA 13: EXPORTAR SI APROBADO
# ============================================================================

if INCLUDE_IN_ENSEMBLE:
    print("\n" + "="*80)
    print("💾 EXPORTANDO PREDICCIONES")
    print("="*80)
    
    # Convertir a retornos para meta-learner
    y_prev_levels = holdout_df['PEN_lag_1'].values
    pred_returns = np.log(FINAL_PREDICTIONS / y_prev_levels)
    
    # DataFrame de exportación
    export_df = pd.DataFrame({
        'ds': holdout_df.index.strftime('%Y-%m-%d'),
        'y_pred': pred_returns,
        'model': 'Chronos2',
        'type': 'log_returns'
    })
    
    pred_path = PRED_DUMP_DIR / 'pred_Chronos2.csv'
    export_df.to_csv(pred_path, index=False)
    print(f"✅ Predicciones guardadas: {pred_path}")
    
    # Guardar métricas
    metrics_output = {
        'model': 'Chronos2',
        'strategy': FINAL_STRATEGY,
        'DA': best_results['DA'],
        'MASE': best_results['MASE'],
        'timestamp': datetime.now().isoformat()
    }
    
    metrics_path = OUTPUT_DIR / 'metrics.json'
    with open(metrics_path, 'w') as f:
        json.dump(metrics_output, f, indent=2)
    print(f"✅ Métricas guardadas: {metrics_path}")

else:
    print("\n⚠️ No se exportan predicciones - modelo descartado")
    
    # Guardar registro de descarte
    discard_info = {
        'model': 'Chronos2',
        'status': 'DESCARTADO',
        'reason': f'DA máximo ({best_results["DA"]:.2f}%) < umbral ({THRESHOLD_DA}%)',
        'best_strategy': best_strategy,
        'all_results': {k: {'DA': v['DA'], 'MASE': v['MASE']} for k, v in all_results.items()},
        'timestamp': datetime.now().isoformat()
    }
    
    discard_path = OUTPUT_DIR / 'chronos2_discarded.json'
    with open(discard_path, 'w') as f:
        json.dump(discard_info, f, indent=2)
    print(f"📄 Registro de descarte: {discard_path}")


⚠️ No se exportan predicciones - modelo descartado
📄 Registro de descarte: C:\Users\Carlos Palma\OneDrive\Documents\Cursos\UTEC Computer Science\TESIS\NUEVO PAPER\tesis_maestria\Chronos2_h1_USD_PEN\chronos2_discarded.json


In [16]:
# ============================================================================
# CELDA 14: RESUMEN FINAL
# ============================================================================

print("\n" + "="*80)
print("🏁 CHRONOS-2 USD/PEN - RESUMEN FINAL")
print("="*80)

print(f"""
📊 RESULTADOS DEL ABLATION STUDY:
""")

for name, res in sorted(all_results.items(), key=lambda x: -x[1]['DA']):
    status = "🏆" if name == best_strategy else "  "
    print(f"   {status} {name:<20}: DA={res['DA']:.2f}%, MASE={res['MASE']:.4f}")

print(f"""
📋 DECISIÓN:
   Estado:     {'✅ INCLUIDO EN ENSEMBLE' if INCLUDE_IN_ENSEMBLE else '❌ DESCARTADO'}
   Estrategia: {FINAL_STRATEGY if FINAL_STRATEGY else 'N/A'}
   Umbral DA:  {THRESHOLD_DA}%

📁 ARCHIVOS:
   Output dir: {OUTPUT_DIR}
""")

if not INCLUDE_IN_ENSEMBLE:
    print("""
💡 NOTA:
   Chronos-2 no logra resultados aceptables para USD/PEN.
   Esto es consistente con la hipótesis de que los foundation models
   entrenados con series de negocio (ventas, demanda) no funcionan
   bien con series financieras (tipo de cambio).
   
   Recomendación: Usar los otros 6-7 modelos del ensemble.
""")


🏁 CHRONOS-2 USD/PEN - RESUMEN FINAL

📊 RESULTADOS DEL ABLATION STUDY:

   🏆 detrended           : DA=46.67%, MASE=1.4117
      log_level           : DA=45.00%, MASE=1.4178
      bias_corrected      : DA=41.67%, MASE=1.4447
      baseline            : DA=40.00%, MASE=1.4113

📋 DECISIÓN:
   Estado:     ❌ DESCARTADO
   Estrategia: N/A
   Umbral DA:  48.0%

📁 ARCHIVOS:
   Output dir: C:\Users\Carlos Palma\OneDrive\Documents\Cursos\UTEC Computer Science\TESIS\NUEVO PAPER\tesis_maestria\Chronos2_h1_USD_PEN


💡 NOTA:
   Chronos-2 no logra resultados aceptables para USD/PEN.
   Esto es consistente con la hipótesis de que los foundation models
   entrenados con series de negocio (ventas, demanda) no funcionan
   bien con series financieras (tipo de cambio).

   Recomendación: Usar los otros 6-7 modelos del ensemble.



In [ ]:
# %%
# ================================================================================
# CELDA 4: FEATURE ENGINEERING CON ANTI-LEAKAGE
# ================================================================================

class FeatureEngineer:
    """
    Feature Engineering con ANTI-LEAKAGE ESTRICTO.
    
    PRINCIPIO: En el momento t, solo usamos información hasta t-1.
    - Todas las features calculadas usan .shift(1)
    - Features técnicas (RSI, MACD, GARCH) se calculan sobre datos ya lagged
    """
    
    def __init__(self, target_col: str = 'PEN'):
        self.target_col = target_col
        self.scaler = StandardScaler()
        self.feature_cols = []
    
    def calculate_log_returns(self, df: pd.DataFrame) -> pd.DataFrame:
        """Calcular log returns para target y otras series"""
        df = df.copy()
        
        # Log return del target
        df['PEN_log_ret'] = np.log(df['PEN'] / df['PEN'].shift(1))
        
        # Log returns de otras monedas (con lag para anti-leakage)
        for col in ['MXN', 'CLP', 'COBRE', 'MXPE']:
            if col in df.columns:
                log_ret = np.log(df[col] / df[col].shift(1))
                df[f'{col}_log_ret_lag_1'] = log_ret.shift(1)  # Anti-leakage
        
        return df
    
    def calculate_diff_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Diferencias para features macro (con lag)"""
        df = df.copy()
        
        for col in ['DXY', 'UST10Y']:
            if col in df.columns:
                diff = df[col].diff()
                df[f'{col}_diff_lag_1'] = diff.shift(1)  # Anti-leakage
        
        return df
    
    def calculate_rsi(self, prices: pd.Series, window: int = 14) -> pd.Series:
        """RSI con anti-leakage"""
        # Calcular sobre precios ya lagged
        prices_lagged = prices.shift(1)
        
        delta = prices_lagged.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        
        avg_gain = gain.rolling(window=window, min_periods=window).mean()
        avg_loss = loss.rolling(window=window, min_periods=window).mean()
        
        rs = avg_gain / avg_loss.replace(0, np.nan)
        rsi = 100 - (100 / (1 + rs))
        
        return rsi
    
    def calculate_macd(self, prices: pd.Series, 
                       fast: int = 12, slow: int = 26) -> pd.Series:
        """MACD line con anti-leakage"""
        prices_lagged = prices.shift(1)
        
        ema_fast = prices_lagged.ewm(span=fast, adjust=False).mean()
        ema_slow = prices_lagged.ewm(span=slow, adjust=False).mean()
        
        return ema_fast - ema_slow
    
    def calculate_garch_volatility(self, returns: pd.Series, 
                                   window: int = 252) -> pd.Series:
        """GARCH(1,1) volatility con anti-leakage"""
        returns_lagged = returns.shift(1)
        
        vol = pd.Series(index=returns.index, dtype=float)
        vol[:] = np.nan
        
        for i in range(window, len(returns)):
            try:
                train_returns = returns_lagged.iloc[i-window:i].dropna() * 100
                if len(train_returns) < window // 2:
                    continue
                
                model = arch_model(train_returns, vol='Garch', p=1, q=1, 
                                   rescale=False, mean='Zero')
                result = model.fit(disp='off', show_warning=False)
                
                # Forecast de 1 paso
                forecast = result.forecast(horizon=1)
                vol.iloc[i] = np.sqrt(forecast.variance.values[-1, 0]) / 100
                
            except:
                vol.iloc[i] = returns_lagged.iloc[i-window:i].std()
        
        # Fill inicial con rolling std
        vol = vol.fillna(returns_lagged.rolling(window=21).std())
        
        return vol
    
    def calculate_temporal_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Calcular features temporales cíclicas."""
        if 'Dates' in df.columns:
            dates = pd.to_datetime(df['Dates'])  # Asegurar datetime
        else:
            dates = pd.to_datetime(df.index)     # Asegurar datetime
        
        # Mes (cíclico) - usar .dt accessor para Series
        df['month_sin'] = np.sin(2 * np.pi * dates.dt.month / 12)
        df['month_cos'] = np.cos(2 * np.pi * dates.dt.month / 12)
        
        # Día de la semana (cíclico) - usar .dt accessor
        df['day_of_week_sin'] = np.sin(2 * np.pi * dates.dt.dayofweek / 7)
        df['day_of_week_cos'] = np.cos(2 * np.pi * dates.dt.dayofweek / 7)
        
        return df
    
    def engineer_all_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Pipeline completo de feature engineering"""
        logger.info("Iniciando feature engineering con anti-leakage...")
        
        df = df.copy()
        
        # 1. Log returns
        df = self.calculate_log_returns(df)
        logger.info("  ✓ Log returns calculados")
        
        # 2. Diferencias macro
        df = self.calculate_diff_features(df)
        logger.info("  ✓ Diferencias macro calculadas")
        
        # 3. RSI (sobre precios lagged)
        df['RSI'] = self.calculate_rsi(df['PEN'])
        logger.info("  ✓ RSI calculado")
        
        # 4. MACD (sobre precios lagged)
        df['MACD_line'] = self.calculate_macd(df['PEN'])
        logger.info("  ✓ MACD calculado")
        
        # 5. GARCH volatility (sobre returns lagged)
        df['GARCH_vol'] = self.calculate_garch_volatility(df['PEN_log_ret'])
        logger.info("  ✓ GARCH volatility calculado")
        
        # 6. Features temporales
        df = self.calculate_temporal_features(df)
        logger.info("  ✓ Features temporales calculadas")
        
        # Definir columnas de features exógenas
        self.feature_cols = [
            # Técnicas (3)
            'RSI', 'MACD_line', 'GARCH_vol',
            # Macro/FX con lag (6)
            'MXN_log_ret_lag_1', 'CLP_log_ret_lag_1', 'COBRE_log_ret_lag_1',
            'MXPE_log_ret_lag_1', 'DXY_diff_lag_1', 'UST10Y_diff_lag_1',
            # Temporales (4)
            'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos'
        ]
        
        # Eliminar NaNs
        initial_len = len(df)
        df = df.dropna(subset=[TARGET_COL] + self.feature_cols)
        final_len = len(df)
        
        logger.info(f"  ✓ NaNs eliminados: {initial_len} → {final_len} ({initial_len - final_len} removidos)")
        logger.info(f"  ✓ Features exógenas: {len(self.feature_cols)}")
        
        return df

logger.info("✓ FeatureEngineer definido")


16:37:16 - INFO - ✅ Datos cargados: 8200 observaciones
16:37:16 - INFO -    Train: 8140 (1994-02-01 00:00:00 a 2025-04-14 00:00:00)
16:37:16 - INFO -    Holdout: 60 (2025-04-15 00:00:00 a 2025-07-07 00:00:00)



📂 CARGANDO DATOS


In [8]:
# %%
# ================================================================================
# CELDA 5: CLASE VISUALIZER PARA PAPER
# ================================================================================

class PaperVisualizer:
    """
    Visualizaciones de calidad para paper académico.
    - Estilo profesional
    - Exportación en alta resolución
    - Tablas en formato LaTeX
    """
    
    def __init__(self, output_dir: Path, figures_dir: Path, tables_dir: Path):
        self.output_dir = output_dir
        self.figures_dir = figures_dir
        self.tables_dir = tables_dir
        
        # Colores consistentes
        self.colors = {
            'real': '#2E86AB',      # Azul
            'pred': '#E94F37',      # Rojo
            'correct': '#28A745',   # Verde
            'incorrect': '#DC3545', # Rojo
            'baseline': '#6C757D',  # Gris
            'chronos': '#9B59B6',   # Púrpura
        }
    
    def plot_predictions_vs_real(self, dates: list, y_true: np.ndarray, 
                                  y_pred: np.ndarray, title: str,
                                  filename: str) -> None:
        """Gráfico de predicciones vs valores reales"""
        fig, axes = plt.subplots(2, 1, figsize=(14, 8), 
                                  gridspec_kw={'height_ratios': [2, 1]})
        
        # Panel superior: Series temporales
        ax1 = axes[0]
        ax1.plot(dates, y_true, 'o-', color=self.colors['real'], 
                 label='Real', alpha=0.8, markersize=4, linewidth=1.5)
        ax1.plot(dates, y_pred, 's-', color=self.colors['pred'], 
                 label='Predicted', alpha=0.8, markersize=4, linewidth=1.5)
        ax1.axhline(y=0, color='black', linestyle='--', linewidth=0.8, alpha=0.5)
        ax1.set_ylabel('Log Return')
        ax1.set_title(title, fontsize=14, fontweight='bold')
        ax1.legend(loc='upper right')
        ax1.grid(True, alpha=0.3)
        
        # Formatear fechas
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        ax1.xaxis.set_major_locator(mdates.WeekdayLocator(interval=2))
        plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')
        
        # Panel inferior: Errores
        ax2 = axes[1]
        errors = y_true - y_pred
        colors_bar = [self.colors['correct'] if e >= 0 else self.colors['incorrect'] 
                      for e in errors]
        ax2.bar(dates, errors, color=colors_bar, alpha=0.7, width=0.8)
        ax2.axhline(y=0, color='black', linewidth=0.8)
        ax2.set_ylabel('Error (Real - Pred)')
        ax2.set_xlabel('Date')
        ax2.grid(True, alpha=0.3)
        
        ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        ax2.xaxis.set_major_locator(mdates.WeekdayLocator(interval=2))
        plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')
        
        plt.tight_layout()
        
        path = self.figures_dir / filename
        plt.savefig(path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        logger.info(f"✓ Saved: {path}")
    
    def plot_directional_accuracy(self, dates: list, y_true: np.ndarray,
                                   y_pred: np.ndarray, title: str,
                                   filename: str) -> None:
        """Gráfico de accuracy direccional día a día"""
        fig, ax = plt.subplots(figsize=(14, 6))
        
        # Calcular aciertos
        true_dir = np.sign(y_true)
        pred_dir = np.sign(y_pred)
        correct = (true_dir == pred_dir).astype(int)
        
        # Barras de acierto/error
        colors_bar = [self.colors['correct'] if c == 1 else self.colors['incorrect'] 
                      for c in correct]
        ax.bar(dates, correct, color=colors_bar, alpha=0.7, width=0.8)
        
        # DA acumulativo
        cum_da = np.cumsum(correct) / (np.arange(len(correct)) + 1) * 100
        ax2 = ax.twinx()
        ax2.plot(dates, cum_da, 'b-', linewidth=2.5, label='Cumulative DA')
        ax2.axhline(y=50, color='red', linestyle='--', linewidth=1.5, 
                    label='Random Walk (50%)')
        ax2.set_ylabel('Cumulative DA (%)', color='blue')
        ax2.tick_params(axis='y', labelcolor='blue')
        ax2.set_ylim(30, 70)
        
        ax.set_ylabel('Correct (1) / Incorrect (0)')
        ax.set_xlabel('Date')
        ax.set_title(title, fontsize=14, fontweight='bold')
        
        # Leyenda combinada
        legend_elements = [
            Patch(facecolor=self.colors['correct'], alpha=0.7, label='Correct'),
            Patch(facecolor=self.colors['incorrect'], alpha=0.7, label='Incorrect'),
            Line2D([0], [0], color='blue', linewidth=2.5, label='Cumulative DA'),
            Line2D([0], [0], color='red', linestyle='--', linewidth=1.5, label='Random Walk')
        ]
        ax.legend(handles=legend_elements, loc='upper left')
        
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=2))
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
        
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        path = self.figures_dir / filename
        plt.savefig(path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        logger.info(f"✓ Saved: {path}")
    
    def plot_model_comparison(self, results: Dict, filename: str) -> None:
        """Comparación de todos los modelos (DA y MASE)"""
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        models = list(results.keys())
        da_values = [results[m]['DA'] for m in models]
        mase_values = [results[m]['MASE'] for m in models]
        
        # Colores: resaltar Chronos-2
        colors_da = [self.colors['chronos'] if 'Chronos' in m else self.colors['baseline'] 
                     for m in models]
        colors_mase = colors_da.copy()
        
        # Panel DA
        ax1 = axes[0]
        bars1 = ax1.barh(models, da_values, color=colors_da, alpha=0.8)
        ax1.axvline(x=50, color='red', linestyle='--', linewidth=2, 
                    label='Random Walk (50%)')
        ax1.set_xlabel('Directional Accuracy (%)')
        ax1.set_title('DA Comparison', fontsize=13, fontweight='bold')
        ax1.legend(loc='lower right')
        ax1.grid(True, alpha=0.3, axis='x')
        ax1.set_xlim(35, 60)
        
        # Añadir valores
        for bar, val in zip(bars1, da_values):
            ax1.text(val + 0.5, bar.get_y() + bar.get_height()/2, 
                     f'{val:.1f}%', va='center', fontsize=10)
        
        # Panel MASE
        ax2 = axes[1]
        bars2 = ax2.barh(models, mase_values, color=colors_mase, alpha=0.8)
        ax2.axvline(x=1.0, color='red', linestyle='--', linewidth=2, 
                    label='Naive Benchmark (1.0)')
        ax2.set_xlabel('MASE')
        ax2.set_title('MASE Comparison', fontsize=13, fontweight='bold')
        ax2.legend(loc='lower right')
        ax2.grid(True, alpha=0.3, axis='x')
        
        # Añadir valores
        for bar, val in zip(bars2, mase_values):
            ax2.text(val + 0.02, bar.get_y() + bar.get_height()/2, 
                     f'{val:.4f}', va='center', fontsize=10)
        
        plt.tight_layout()
        path = self.figures_dir / filename
        plt.savefig(path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        logger.info(f"✓ Saved: {path}")
    
    def plot_ablation_study(self, ablation_results: Dict, filename: str) -> None:
        """Gráfico de ablation study (univariado vs multivariado)"""
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        
        experiments = list(ablation_results.keys())
        da_values = [ablation_results[e]['DA'] for e in experiments]
        mase_values = [ablation_results[e]['MASE'] for e in experiments]
        
        colors = ['#3498DB', '#E74C3C', '#2ECC71'][:len(experiments)]
        
        # DA
        ax1 = axes[0]
        bars1 = ax1.bar(experiments, da_values, color=colors, alpha=0.8, edgecolor='black')
        ax1.axhline(y=50, color='red', linestyle='--', linewidth=2, label='Random Walk')
        ax1.set_ylabel('Directional Accuracy (%)')
        ax1.set_title('Ablation Study: DA', fontsize=13, fontweight='bold')
        ax1.legend()
        ax1.set_ylim(40, 60)
        ax1.grid(True, alpha=0.3, axis='y')
        
        for bar, val in zip(bars1, da_values):
            ax1.text(bar.get_x() + bar.get_width()/2, val + 0.5, 
                     f'{val:.2f}%', ha='center', fontsize=11, fontweight='bold')
        
        # MASE
        ax2 = axes[1]
        bars2 = ax2.bar(experiments, mase_values, color=colors, alpha=0.8, edgecolor='black')
        ax2.axhline(y=1.0, color='red', linestyle='--', linewidth=2, label='Naive Benchmark')
        ax2.set_ylabel('MASE')
        ax2.set_title('Ablation Study: MASE', fontsize=13, fontweight='bold')
        ax2.legend()
        ax2.grid(True, alpha=0.3, axis='y')
        
        for bar, val in zip(bars2, mase_values):
            ax2.text(bar.get_x() + bar.get_width()/2, val + 0.02, 
                     f'{val:.4f}', ha='center', fontsize=11, fontweight='bold')
        
        plt.tight_layout()
        path = self.figures_dir / filename
        plt.savefig(path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        logger.info(f"✓ Saved: {path}")
    
    def generate_latex_table(self, results: Dict, caption: str, 
                             label: str, filename: str) -> str:
        """Generar tabla en formato LaTeX"""
        
        latex = r"""
\begin{table}[htbp]
\centering
\caption{""" + caption + r"""}
\label{""" + label + r"""}
\begin{tabular}{lcccc}
\toprule
\textbf{Model} & \textbf{DA (\%)} & \textbf{MASE} & \textbf{$\Delta$DA vs RW} & \textbf{$\Delta$MASE vs Naive} \\
\midrule
"""
        
        for model, metrics in results.items():
            da = metrics['DA']
            mase = metrics['MASE']
            delta_da = da - 50  # vs Random Walk
            delta_mase = mase - 1.0  # vs Naive
            
            # Formato con signo
            delta_da_str = f"+{delta_da:.2f}" if delta_da > 0 else f"{delta_da:.2f}"
            delta_mase_str = f"+{delta_mase:.4f}" if delta_mase > 0 else f"{delta_mase:.4f}"
            
            # Bold si es mejor que baseline
            if 'Chronos' in model:
                model_str = r"\textbf{" + model + "}"
                da_str = r"\textbf{" + f"{da:.2f}" + "}"
                mase_str = r"\textbf{" + f"{mase:.4f}" + "}"
            else:
                model_str = model
                da_str = f"{da:.2f}"
                mase_str = f"{mase:.4f}"
            
            latex += f"{model_str} & {da_str} & {mase_str} & {delta_da_str} & {delta_mase_str} \\\\\n"
        
        latex += r"""
\bottomrule
\end{tabular}
\end{table}
"""
        
        # Guardar
        path = self.tables_dir / filename
        with open(path, 'w') as f:
            f.write(latex)
        logger.info(f"✓ Saved: {path}")
        
        return latex
    
    def generate_metrics_csv(self, results: Dict, filename: str) -> pd.DataFrame:
        """Exportar métricas a CSV para análisis posterior"""
        
        rows = []
        for model, metrics in results.items():
            rows.append({
                'Model': model,
                'DA': metrics['DA'],
                'MASE': metrics['MASE'],
                'MAE': metrics.get('MAE', np.nan),
                'RMSE': metrics.get('RMSE', np.nan),
                'Delta_DA_vs_RW': metrics['DA'] - 50,
                'Delta_MASE_vs_Naive': metrics['MASE'] - 1.0
            })
        
        df = pd.DataFrame(rows)
        path = self.tables_dir / filename
        df.to_csv(path, index=False)
        logger.info(f"✓ Saved: {path}")
        
        return df

logger.info("✓ PaperVisualizer definido")


23:35:55 - INFO - ✓ PaperVisualizer definido


In [22]:
# %%
# ================================================================================
# CELDA 6: CARGAR Y PREPARAR DATOS
# ================================================================================

logger.info("=" * 80)
logger.info("CARGANDO DATOS")
logger.info("=" * 80)

# Cargar
df_raw = pd.read_csv(DATA_PATH)
logger.info(f"Raw data: {df_raw.shape}")

# Convertir fechas
df_raw['Dates'] = pd.to_datetime(df_raw['Dates'])
df_raw = df_raw.sort_values('Dates').reset_index(drop=True)

logger.info(f"Rango: {df_raw['Dates'].min()} → {df_raw['Dates'].max()}")

# Feature Engineering
engineer = FeatureEngineer(target_col='PEN')
df_features = engineer.engineer_all_features(df_raw)

# Set index
df_features = df_features.set_index('Dates')

# Verificar features
EXOG_FEATURES = engineer.feature_cols
logger.info(f"\nFeatures exógenas ({len(EXOG_FEATURES)}):")
for i, f in enumerate(EXOG_FEATURES):
    logger.info(f"  {i+1}. {f}")

logger.info(f"\nTotal observaciones: {len(df_features)}")

01:50:59 - INFO - ================================================================================
01:50:59 - INFO - CARGANDO DATOS
01:50:59 - INFO - ================================================================================
01:50:59 - INFO - Raw data: (8201, 12)
01:50:59 - INFO - Rango: 1994-01-31 00:00:00 → 2025-07-07 00:00:00
01:50:59 - INFO - Iniciando feature engineering con anti-leakage...
01:50:59 - INFO -   ✓ Log returns calculados
01:50:59 - INFO -   ✓ Diferencias macro calculadas
01:50:59 - INFO -   ✓ RSI calculado
01:50:59 - INFO -   ✓ MACD calculado
01:52:13 - INFO -   ✓ GARCH volatility calculado
01:52:13 - INFO -   ✓ Features temporales calculadas
01:52:13 - INFO -   ✓ NaNs eliminados: 8201 → 8172 (29 removidos)
01:52:13 - INFO -   ✓ Features exógenas: 13
01:52:13 - INFO - 
Features exógenas (13):
01:52:13 - INFO -   1. RSI
01:52:13 - INFO -   2. MACD_line
01:52:13 - INFO -   3. GARCH_vol
01:52:13 - INFO -   4. MXN_log_ret_lag_1
01:52:13 - INFO -   5. CLP_log_ret_la

In [10]:
# %%
# ================================================================================
# CELDA 7: SPLIT TRAIN/HOLDOUT
# ================================================================================

logger.info("=" * 80)
logger.info("SPLIT TRAIN/HOLDOUT")
logger.info("=" * 80)

# Split: últimos N_HOLDOUT días para holdout
n_total = len(df_features)
n_train = n_total - N_HOLDOUT

train_df = df_features.iloc[:n_train].copy()
holdout_df = df_features.iloc[n_train:].copy()

# Guardar valores de niveles para referencia
train_levels = train_df['PEN'].values
holdout_levels = holdout_df['PEN'].values

# Ground truth
y_train = train_df[TARGET_COL].values
y_holdout = holdout_df[TARGET_COL].values

logger.info(f"Train: {len(train_df)} obs ({train_df.index[0]} → {train_df.index[-1]})")
logger.info(f"Holdout: {len(holdout_df)} obs ({holdout_df.index[0]} → {holdout_df.index[-1]})")
logger.info(f"✓ Holdout BLOQUEADO - No se usará para entrenamiento")

# Full dataset para rolling forecast
full_df = pd.concat([train_df, holdout_df])

23:42:31 - INFO - ================================================================================
23:42:31 - INFO - SPLIT TRAIN/HOLDOUT
23:42:31 - INFO - ================================================================================
23:42:31 - INFO - Train: 8112 obs (1994-03-02 00:00:00 → 2025-04-14 00:00:00)
23:42:31 - INFO - Holdout: 60 obs (2025-04-15 00:00:00 → 2025-07-07 00:00:00)
23:42:31 - INFO - ✓ Holdout BLOQUEADO - No se usará para entrenamiento


In [14]:
# %%
# ================================================================================
# CELDA 8: CLASE CHRONOS-2 FORECASTER
# ================================================================================

class Chronos2Forecaster:
    """
    Forecaster usando Chronos-2 con soporte nativo de covariates.
    
    Características:
    - API simple basada en DataFrames
    - Soporte univariado y multivariado
    - Rolling forecast sin reentrenamiento
    - Anti-leakage verificado
    """
    
    def __init__(self, model_id: str = "amazon/chronos-2", device: str = "cuda"):
        """Inicializar Chronos-2 pipeline"""
        self.model_id = model_id
        self.device = device if torch.cuda.is_available() else "cpu"
        self.pipeline = None
        self.is_loaded = False
    
    def load_model(self):
        """Cargar modelo (lazy loading)"""
        if not self.is_loaded:
            logger.info(f"⏳ Cargando Chronos-2 ({self.model_id})...")
            self.pipeline = Chronos2Pipeline.from_pretrained(
                self.model_id,
                device_map=self.device
            )
            self.is_loaded = True
            logger.info(f"✓ Chronos-2 cargado en {self.device}")
    
    def prepare_context_df(self, context_df: pd.DataFrame, 
                           target_col: str,
                           covariate_cols: List[str] = None) -> pd.DataFrame:
        """
        Preparar DataFrame para Chronos-2.
        
        Chronos-2 espera:
        - id: identificador de serie
        - timestamp: fechas
        - target: valores a predecir
        - [covariates]: columnas adicionales (opcional)
        """
        df = context_df.asfreq('B').ffill().reset_index()
        
        # Renombrar columnas según API
        df = df.rename(columns={
            context_df.index.name or 'Dates': 'timestamp',
            target_col: 'target'
        })
        
        # Agregar id
        df['id'] = 'USDPEN'
        
        # Seleccionar columnas
        cols = ['id', 'timestamp', 'target']
        if covariate_cols:
            cols.extend(covariate_cols)
        
        return df[cols].copy()
    
    def rolling_forecast(self, train_df: pd.DataFrame,
                         holdout_df: pd.DataFrame,
                         target_col: str,
                         covariate_cols: List[str] = None,
                         use_covariates: bool = False) -> Dict:
        """
        Rolling forecast h=1 sin reentrenamiento.
        
        Metodología:
        1. Modelo se usa en modo zero-shot (no se reentrena)
        2. En cada paso, usamos valores reales observados como contexto
        3. NO hay leakage porque no aprendemos del holdout
        
        Args:
            train_df: DataFrame de entrenamiento
            holdout_df: DataFrame de holdout (60 días)
            target_col: Columna target
            covariate_cols: Lista de covariates (None para univariado)
            use_covariates: Si True, incluir covariates
        
        Returns:
            Dict con predicciones, métricas y metadata
        """
        self.load_model()
        
        n_holdout = len(holdout_df)
        predictions = []
        
        mode = "Multivariado" if use_covariates and covariate_cols else "Univariado"
        logger.info(f"\n🔄 Rolling forecast ({mode})...")
        logger.info(f"   Horizonte: h={H_FORECAST}")
        logger.info(f"   Holdout: {n_holdout} días")
        if use_covariates and covariate_cols:
            logger.info(f"   Covariates: {len(covariate_cols)}")
        
        full_df = pd.concat([train_df, holdout_df])
        
        for i in range(n_holdout):
            if i % 10 == 0:
                logger.info(f"   Día {i+1}/{n_holdout}...")
            
            # Contexto: train + holdout[:i]
            context_end = len(train_df) + i
            context_df = full_df.iloc[:context_end].copy()
            
            # Preparar para Chronos-2
            if use_covariates and covariate_cols:
                input_df = self.prepare_context_df(context_df, target_col, covariate_cols)
            else:
                input_df = self.prepare_context_df(context_df, target_col, None)
            
            try:
                # Predicción
                pred_df = self.pipeline.predict_df(
                    input_df,
                    prediction_length=1,
                    quantile_levels=[0.5],  # Mediana
                    id_column='id',
                    timestamp_column='timestamp',
                    target='target'
                )
                
                # Extraer predicción (mediana = predictions column)
                pred = float(pred_df['predictions'].iloc[0])
                predictions.append(pred)
                
            except Exception as e:
                logger.warning(f"   ⚠️ Error día {i}: {e}")
                predictions.append(0.0)
        
        predictions = np.array(predictions)
        
        # Calcular métricas
        y_true = holdout_df[target_col].values
        
        da = FXMetrics.calculate_da(y_true, predictions)
        mase = FXMetrics.calculate_mase(y_true, predictions, train_df[target_col].values)
        mae = FXMetrics.calculate_mae(y_true, predictions)
        rmse = FXMetrics.calculate_rmse(y_true, predictions)
        
        # Verificar zeros
        n_zeros = np.sum(predictions == 0)
        if n_zeros > 0:
            logger.warning(f"   ⚠️ {n_zeros} predicciones son 0.0")
        
        results = {
            'predictions': predictions,
            'y_true': y_true,
            'dates': holdout_df.index.tolist(),
            'DA': round(da, 2),
            'MASE': round(mase, 4),
            'MAE': round(mae, 6),
            'RMSE': round(rmse, 6),
            'mode': mode,
            'use_covariates': use_covariates,
            'n_covariates': len(covariate_cols) if covariate_cols else 0,
            'n_zeros': n_zeros
        }
        
        logger.info(f"\n✓ Resultados ({mode}):")
        logger.info(f"   DA:   {da:.2f}%")
        logger.info(f"   MASE: {mase:.4f}")
        logger.info(f"   MAE:  {mae:.6f}")
        
        return results
    
    def generate_oof_predictions(self, train_df: pd.DataFrame,
                                  target_col: str,
                                  covariate_cols: List[str] = None,
                                  use_covariates: bool = False,
                                  checkpoint_file: Path = None) -> Dict:
        """
        Generar predicciones Out-of-Fold usando Walk-Forward CV.
        
        Para el META-LEARNER:
        - Genera predicciones en todo el train set
        - Usa expanding window (MIN_TRAIN inicial, incrementa STEP_SIZE)
        - Checkpointing cada 5 folds
        
        Args:
            train_df: DataFrame de entrenamiento
            target_col: Columna target
            covariate_cols: Lista de covariates
            use_covariates: Si True, incluir covariates
            checkpoint_file: Path para checkpoint (opcional)
        
        Returns:
            Dict con predicciones OOF, métricas y metadata
        """
        self.load_model()
        
        if checkpoint_file is None:
            checkpoint_file = CHECKPOINT_DIR / 'oof_chronos2_checkpoint.pkl'
        
        mode = "Multivariado" if use_covariates and covariate_cols else "Univariado"
        
        logger.info(f"\n{'='*80}")
        logger.info(f"🔄 GENERANDO OOF ({mode}) - WALK-FORWARD CV")
        logger.info(f"{'='*80}")
        
        # Verificar checkpoint
        resume = False
        if checkpoint_file.exists():
            try:
                with open(checkpoint_file, 'rb') as f:
                    checkpoint = pickle.load(f)
                logger.info(f"📂 Checkpoint encontrado: fold {checkpoint['last_fold']}")
                resume = True
            except:
                resume = False
        
        # Inicializar estado
        if resume:
            oof_preds = checkpoint['oof_predictions']
            oof_dates = checkpoint['oof_dates']
            oof_actuals = checkpoint['oof_actuals']
            fold_metrics = checkpoint['fold_metrics']
            start_fold = checkpoint['last_fold'] + 1
            cumulative_time = checkpoint['elapsed_seconds']
        else:
            oof_preds = []
            oof_dates = []
            oof_actuals = []
            fold_metrics = []
            start_fold = 1
            cumulative_time = 0
        
        # Configuración
        n_obs = len(train_df)
        n_splits = (n_obs - MIN_TRAIN) // STEP_SIZE
        
        logger.info(f"\n📊 Configuración:")
        logger.info(f"   Observaciones: {n_obs}")
        logger.info(f"   Min train: {MIN_TRAIN}")
        logger.info(f"   Step size: {STEP_SIZE}")
        logger.info(f"   Splits: {n_splits}")
        
        start_time = datetime.now()
        
        # Walk-forward loop
        fold = 0
        positions = list(range(MIN_TRAIN, n_obs, STEP_SIZE))
        
        for start in positions:
            fold += 1
            
            if fold < start_fold:
                continue
            
            end = min(start + STEP_SIZE, n_obs)
            val_indices = list(range(start, end))
            
            if len(val_indices) == 0:
                continue
            
            logger.info(f"\n{'─'*50}")
            logger.info(f"Fold {fold}/{n_splits} | Context: 0→{start} | Val: {start}→{end}")
            
            fold_preds_list = []
            fold_true_list = []
            fold_dates_list = []
            n_errors = 0
            
            for idx in val_indices:
                # Contexto hasta idx
                context_df = train_df.iloc[:idx].copy()
                
                # Preparar input
                if use_covariates and covariate_cols:
                    input_df = self.prepare_context_df(context_df, target_col, covariate_cols)
                else:
                    input_df = self.prepare_context_df(context_df, target_col, None)
                
                try:
                    pred_df = self.pipeline.predict_df(
                        input_df,
                        prediction_length=1,
                        quantile_levels=[0.5],
                        id_column='id',
                        timestamp_column='timestamp',
                        target='target'
                    )
                    pred = float(pred_df['predictions'].iloc[0])
                except Exception as e:
                    if n_errors < 3:
                        logger.warning(f"   Error idx {idx}: {e}")
                    n_errors += 1
                    pred = 0.0
                
                true_val = train_df.iloc[idx][target_col]
                date_val = train_df.index[idx]
                
                fold_preds_list.append(pred)
                fold_true_list.append(true_val)
                fold_dates_list.append(date_val)
            
            fold_preds = np.array(fold_preds_list)
            fold_true = np.array(fold_true_list)
            
            # Métricas del fold
            n_zeros = np.sum(fold_preds == 0)
            fold_da = FXMetrics.calculate_da(fold_true, fold_preds) if n_zeros < len(fold_preds) else 0.0
            fold_mae = np.mean(np.abs(fold_true - fold_preds))
            
            status = "✅" if n_zeros == 0 else f"⚠️ ({n_zeros} zeros)"
            logger.info(f"   {status} DA: {fold_da:.1f}% | MAE: {fold_mae:.6f}")
            
            # Guardar
            oof_preds.extend(fold_preds_list)
            oof_dates.extend(fold_dates_list)
            oof_actuals.extend(fold_true_list)
            
            fold_metrics.append({
                'fold': fold,
                'context_size': start,
                'val_size': len(val_indices),
                'da': float(fold_da),
                'mae': float(fold_mae),
                'n_zeros': int(n_zeros)
            })
            
            # Checkpoint cada 5 folds
            if fold % 5 == 0:
                elapsed = cumulative_time + (datetime.now() - start_time).total_seconds()
                
                with open(checkpoint_file, 'wb') as f:
                    pickle.dump({
                        'last_fold': fold,
                        'oof_predictions': oof_preds,
                        'oof_dates': oof_dates,
                        'oof_actuals': oof_actuals,
                        'fold_metrics': fold_metrics,
                        'elapsed_seconds': elapsed,
                        'mode': mode
                    }, f)
                
                logger.info(f"   💾 Checkpoint ({elapsed/60:.1f} min)")
        
        # Métricas finales
        oof_preds = np.array(oof_preds)
        oof_actuals = np.array(oof_actuals)
        
        elapsed_total = cumulative_time + (datetime.now() - start_time).total_seconds()
        
        n_zeros = np.sum(oof_preds == 0)
        da_oof = FXMetrics.calculate_da(oof_actuals, oof_preds)
        mase_oof = FXMetrics.calculate_mase(oof_actuals, oof_preds, y_train)
        mae_oof = np.mean(np.abs(oof_actuals - oof_preds))
        
        logger.info(f"\n{'='*60}")
        logger.info(f"📊 RESUMEN OOF ({mode})")
        logger.info(f"{'='*60}")
        logger.info(f"   Observaciones: {len(oof_preds)}")
        logger.info(f"   Zeros: {n_zeros}")
        logger.info(f"   DA OOF: {da_oof:.2f}%")
        logger.info(f"   MASE OOF: {mase_oof:.4f}")
        logger.info(f"   Tiempo: {elapsed_total/60:.1f} min")
        
        # Eliminar checkpoint
        if checkpoint_file.exists():
            checkpoint_file.unlink()
        
        return {
            'predictions': oof_preds,
            'dates': oof_dates,
            'actuals': oof_actuals,
            'DA': round(da_oof, 2),
            'MASE': round(mase_oof, 4),
            'MAE': round(mae_oof, 6),
            'mode': mode,
            'n_zeros': n_zeros,
            'n_observations': len(oof_preds),
            'elapsed_minutes': elapsed_total / 60,
            'fold_metrics': fold_metrics
        }

logger.info("✓ Chronos2Forecaster definido")


00:54:28 - INFO - ✓ Chronos2Forecaster definido


In [16]:
# %%
# ================================================================================
# CELDA 9: EJECUTAR ABLATION STUDY (HOLDOUT)
# ================================================================================

logger.info("=" * 80)
logger.info("ABLATION STUDY: UNIVARIADO vs MULTIVARIADO")
logger.info("=" * 80)

# Inicializar forecaster
forecaster = Chronos2Forecaster(model_id=CHRONOS_MODEL_ID, device=DEVICE)

# Almacenar resultados
ablation_results = {}

# ───────────────────────────────────────────────────────────────────────────────
# EXPERIMENTO 1: UNIVARIADO (sin covariates)
# ───────────────────────────────────────────────────────────────────────────────

logger.info("\n" + "─" * 60)
logger.info("EXPERIMENTO 1: UNIVARIADO (Zero-Shot)")
logger.info("─" * 60)

results_uni = forecaster.rolling_forecast(
    train_df=train_df,
    holdout_df=holdout_df,
    target_col=TARGET_COL,
    covariate_cols=None,
    use_covariates=False
)

ablation_results['Univariado'] = {
    'DA': results_uni['DA'],
    'MASE': results_uni['MASE'],
    'MAE': results_uni['MAE'],
    'predictions': results_uni['predictions'],
    'config': {'covariates': 0}
}

# ───────────────────────────────────────────────────────────────────────────────
# EXPERIMENTO 2: MULTIVARIADO (con 13 exógenas)
# ───────────────────────────────────────────────────────────────────────────────

logger.info("\n" + "─" * 60)
logger.info("EXPERIMENTO 2: MULTIVARIADO (13 Exógenas)")
logger.info("─" * 60)
logger.info(f"Features: {EXOG_FEATURES[:5]}... ({len(EXOG_FEATURES)} total)")

results_multi = forecaster.rolling_forecast(
    train_df=train_df,
    holdout_df=holdout_df,
    target_col=TARGET_COL,
    covariate_cols=EXOG_FEATURES,
    use_covariates=True
)

ablation_results['Multivariado'] = {
    'DA': results_multi['DA'],
    'MASE': results_multi['MASE'],
    'MAE': results_multi['MAE'],
    'predictions': results_multi['predictions'],
    'config': {'covariates': len(EXOG_FEATURES), 'features': EXOG_FEATURES}
}

# ───────────────────────────────────────────────────────────────────────────────
# RESUMEN ABLATION
# ───────────────────────────────────────────────────────────────────────────────

logger.info("\n" + "=" * 60)
logger.info("📊 RESUMEN ABLATION STUDY")
logger.info("=" * 60)
logger.info(f"{'Experimento':<20} {'DA (%)':<12} {'MASE':<12} {'Δ DA vs RW':<12}")
logger.info("-" * 60)

for exp, res in ablation_results.items():
    delta_da = res['DA'] - 50
    delta_str = f"+{delta_da:.2f}" if delta_da > 0 else f"{delta_da:.2f}"
    logger.info(f"{exp:<20} {res['DA']:<12.2f} {res['MASE']:<12.4f} {delta_str:<12}")

logger.info("-" * 60)

# Determinar mejor experimento
best_exp = max(ablation_results.items(), key=lambda x: x[1]['DA'])
best_name = best_exp[0]
best_results = best_exp[1]

logger.info(f"\n🏆 MEJOR EXPERIMENTO: {best_name}")
logger.info(f"   DA: {best_results['DA']:.2f}%")
logger.info(f"   MASE: {best_results['MASE']:.4f}")

# %%
# ════════════════════════════════════════════════════════════════════════════════
# VERIFICAR PROBLEMA DE SIGNO
# ════════════════════════════════════════════════════════════════════════════════

print("\n" + "=" * 70)
print("🔍 VERIFICACIÓN DE SIGNO")
print("=" * 70)

# Usar resultados del mejor experimento
y_real = best_results['y_true']
y_pred = best_results['predictions']

# DA original
dir_real = np.sign(y_real)
dir_pred = np.sign(y_pred)
da_orig = (dir_real == dir_pred).mean() * 100

# DA invertido
dir_pred_inv = -dir_pred
da_inv = (dir_real == dir_pred_inv).mean() * 100

print(f"\n   DA Original:   {da_orig:.2f}%")
print(f"   DA Invertido:  {da_inv:.2f}%")
print(f"   Random Walk:   50.00%")

# Estadísticas
print(f"\n   % Positivos Real: {(dir_real > 0).mean()*100:.1f}%")
print(f"   % Positivos Pred: {(dir_pred > 0).mean()*100:.1f}%")

# Correlación
corr = np.corrcoef(y_real, y_pred)[0, 1]
print(f"\n   Correlación: {corr:.4f}")

# Diagnóstico
if da_inv > 55:
    print(f"\n⚠️ INVERSIÓN DE SIGNO DETECTADA")
    print(f"   Invertir predicciones mejoraría DA a {da_inv:.2f}%")
else:
    print(f"\n❌ No hay inversión de signo")
    print(f"   Chronos-2 simplemente no funciona para USD/PEN")

01:24:28 - INFO - ================================================================================
01:24:28 - INFO - ABLATION STUDY: UNIVARIADO vs MULTIVARIADO
01:24:28 - INFO - ================================================================================
01:24:28 - INFO - 
────────────────────────────────────────────────────────────
01:24:28 - INFO - EXPERIMENTO 1: UNIVARIADO (Zero-Shot)
01:24:28 - INFO - ────────────────────────────────────────────────────────────
01:24:28 - INFO - ⏳ Cargando Chronos-2 (amazon/chronos-2)...
01:24:30 - INFO - ✓ Chronos-2 cargado en cuda
01:24:30 - INFO - 
🔄 Rolling forecast (Univariado)...
01:24:30 - INFO -    Horizonte: h=1
01:24:30 - INFO -    Holdout: 60 días
01:24:30 - INFO -    Día 1/60...
01:24:32 - INFO -    Día 11/60...
01:24:33 - INFO -    Día 21/60...
01:24:35 - INFO -    Día 31/60...
01:24:36 - INFO -    Día 41/60...
01:24:37 - INFO -    Día 51/60...
01:24:39 - INFO - 
✓ Resultados (Univariado):
01:24:39 - INFO -    DA:   36.67%
01:24:39


🔍 VERIFICACIÓN DE SIGNO


KeyError: 'y_true'

In [24]:
# %%
# ════════════════════════════════════════════════════════════════════════════════
# DIAGNÓSTICO COMPLETO - ¿QUÉ ESTÁ PREDICIENDO CHRONOS-2?
# ════════════════════════════════════════════════════════════════════════════════

import numpy as np
import pandas as pd

print("=" * 70)
print("🔬 DIAGNÓSTICO COMPLETO CHRONOS-2")
print("=" * 70)

# Para Windows Local (VS Code)
BASE_DIR = Path(r"C:\Users\Carlos Palma\OneDrive\Documents\Cursos\UTEC Computer Science\TESIS\NUEVO PAPER\tesis_maestria")

# Para Google Colab (descomentar si usas Colab)
# from google.colab import drive
# drive.mount('/content/drive')
# BASE_DIR = Path("/content/drive/MyDrive/Colab_Outputs")

DATA_PATH = BASE_DIR / "DATA.csv"
df = pd.read_csv(DATA_PATH)
df['PEN_log_ret'] = np.log(df['PEN'] / df['PEN'].shift(1))

N_HOLDOUT = 60
y_pred = best_results['predictions']

# Valores del holdout
holdout = df.tail(N_HOLDOUT).copy()
y_real_ret = holdout['PEN_log_ret'].values
y_real_nivel = holdout['PEN'].values
y_prev_nivel = df['PEN'].iloc[-(N_HOLDOUT+1):-1].values  # Niveles del día anterior

print(f"\n📊 ESTADÍSTICAS DE PREDICCIONES:")
print(f"   Predicciones - Mean: {np.mean(y_pred):.6f}, Std: {np.std(y_pred):.6f}")
print(f"   Predicciones - Min:  {np.min(y_pred):.6f}, Max: {np.max(y_pred):.6f}")
print(f"\n   Retornos Real - Mean: {np.mean(y_real_ret):.6f}, Std: {np.std(y_real_ret):.6f}")
print(f"   Retornos Real - Min:  {np.min(y_real_ret):.6f}, Max: {np.max(y_real_ret):.6f}")
print(f"\n   Niveles Real - Mean: {np.mean(y_real_nivel):.4f}, Std: {np.std(y_real_nivel):.4f}")
print(f"   Niveles Real - Min:  {np.min(y_real_nivel):.4f}, Max: {np.max(y_real_nivel):.4f}")

# ¿El modelo predice niveles o retornos?
print("\n" + "=" * 70)
print("🔍 ¿QUÉ ESTÁ PREDICIENDO EL MODELO?")
print("=" * 70)

if np.mean(y_pred) > 1:
    print("   → Predicciones parecen NIVELES (mean > 1)")
    pred_type = "niveles"
else:
    print("   → Predicciones parecen RETORNOS (mean cerca de 0)")
    pred_type = "retornos"

# Tabla comparativa (primeros 10)
print("\n" + "=" * 70)
print("📋 TABLA COMPARATIVA (primeros 15 días)")
print("=" * 70)
print(f"\n{'Día':<5} {'y_pred':<14} {'Ret Real':<14} {'Nivel Real':<12} {'Nivel Prev':<12}")
print("-" * 60)

for i in range(15):
    print(f"{i+1:<5} {y_pred[i]:<14.6f} {y_real_ret[i]:<14.6f} {y_real_nivel[i]:<12.4f} {y_prev_nivel[i]:<12.4f}")

# Calcular DA de múltiples formas
print("\n" + "=" * 70)
print("🎯 DA CALCULADO DE DIFERENTES FORMAS")
print("=" * 70)

# Forma 1: Pred vs Retorno Real (asumiendo pred = retorno)
dir_pred = np.sign(y_pred)
dir_real_ret = np.sign(y_real_ret)
da_1 = (dir_pred == dir_real_ret).mean() * 100

# Forma 2: Invertido
da_2 = ((-dir_pred) == dir_real_ret).mean() * 100

# Forma 3: Si pred es NIVEL, calcular dirección como cambio vs día anterior
dir_pred_from_nivel = np.sign(y_pred - y_prev_nivel)
da_3 = (dir_pred_from_nivel == dir_real_ret).mean() * 100

# Forma 4: Forma 3 invertida
da_4 = ((-dir_pred_from_nivel) == dir_real_ret).mean() * 100

# Forma 5: Predicción desplazada 1 día (lag issue)
dir_pred_shifted = np.sign(np.roll(y_pred, 1))
dir_pred_shifted[0] = dir_pred[0]
da_5 = (dir_pred_shifted == dir_real_ret).mean() * 100

# Forma 6: Predicción desplazada -1 día
dir_pred_shifted_neg = np.sign(np.roll(y_pred, -1))
dir_pred_shifted_neg[-1] = dir_pred[-1]
da_6 = (dir_pred_shifted_neg == dir_real_ret).mean() * 100

print(f"\n   1. Pred (retorno) vs Real:           {da_1:.2f}%")
print(f"   2. Pred INVERTIDO vs Real:           {da_2:.2f}%")
print(f"   3. Pred-Nivel_Prev vs Real:          {da_3:.2f}%")
print(f"   4. -(Pred-Nivel_Prev) vs Real:       {da_4:.2f}%")
print(f"   5. Pred desplazado +1 día vs Real:   {da_5:.2f}%")
print(f"   6. Pred desplazado -1 día vs Real:   {da_6:.2f}%")
print(f"\n   Random Walk:                         50.00%")

# Encontrar el mejor
das = {'Pred=Ret': da_1, 'Invertido': da_2, 'Pred-Prev': da_3, 
       '-(Pred-Prev)': da_4, 'Shift+1': da_5, 'Shift-1': da_6}
best_method = max(das, key=das.get)
best_da = das[best_method]

print(f"\n🏆 MEJOR MÉTODO: {best_method} con DA = {best_da:.2f}%")

if best_da > 55:
    print(f"\n✅ HAY SOLUCIÓN: Aplicar '{best_method}' para corregir")
else:
    print(f"\n❌ Ningún método supera 55% - Chronos-2 no funciona para USD/PEN")

🔬 DIAGNÓSTICO COMPLETO CHRONOS-2

📊 ESTADÍSTICAS DE PREDICCIONES:
   Predicciones - Mean: 0.000024, Std: 0.000037
   Predicciones - Min:  -0.000090, Max: 0.000098

   Retornos Real - Mean: -0.000800, Std: 0.003149
   Retornos Real - Min:  -0.009039, Max: 0.006457

   Niveles Real - Mean: 3.6379, Std: 0.0501
   Niveles Real - Min:  3.5425, Max: 3.7420

🔍 ¿QUÉ ESTÁ PREDICIENDO EL MODELO?
   → Predicciones parecen RETORNOS (mean cerca de 0)

📋 TABLA COMPARATIVA (primeros 15 días)

Día   y_pred         Ret Real       Nivel Real   Nivel Prev  
------------------------------------------------------------
1     0.000098       0.000668       3.7420       3.7395      
2     0.000030       -0.003748      3.7280       3.7420      
3     0.000014       0.000000       3.7280       3.7280      
4     -0.000090      0.000000       3.7280       3.7280      
5     0.000009       -0.005244      3.7085       3.7280      
6     0.000026       -0.002700      3.6985       3.7085      
7     0.000011       -

In [25]:
df_raw['Dates'] = pd.to_datetime(df_raw['Dates'])
df_raw = df_raw.set_index('Dates').sort_index()

print("\n📊 DATOS ORIGINALES (últimas 5 filas de train):")
train_end = len(df_raw) - N_HOLDOUT
print(df_raw['PEN'].iloc[train_end-5:train_end])

print("\n📊 ESCALA DE LOS DATOS:")
print(f"   PEN (niveles) - Mean: {df_raw['PEN'].mean():.4f}")
print(f"   PEN (niveles) - Last: {df_raw['PEN'].iloc[train_end-1]:.4f}")

# ¿Qué debería predecir Chronos-2?
print("\n🎯 LO QUE DEBERÍA PASAR:")
print(f"   Input a Chronos-2: serie de niveles ≈ 3.64")
print(f"   Output de Chronos-2: siguiente nivel ≈ 3.64")
print(f"   Conversión a retorno: log(3.64/3.64) ≈ 0.00")

print("\n❌ LO QUE ESTÁ PASANDO:")
print(f"   Output actual: {y_pred[0]:.6f}")
print(f"   Esto parece un RETORNO, no un NIVEL")

print("\n" + "=" * 70)
print("🔍 POSIBLES CAUSAS:")
print("=" * 70)
print("""
1. Se le está pasando RETORNOS en lugar de NIVELES
   → Chronos-2 predice el siguiente "retorno" (pequeño)
   
2. Se le está pasando NIVELES pero hay una transformación
   interna que convierte a retornos antes de predecir
   
3. El código calcula retorno ANTES de llamar a Chronos-2
   y le pasa eso como target

Necesito ver el código exacto de cómo se preparan los datos.
""")


📊 DATOS ORIGINALES (últimas 5 filas de train):
Dates
2025-04-08    3.7475
2025-04-09    3.6985
2025-04-10    3.7350
2025-04-11    3.7319
2025-04-14    3.7395
Name: PEN, dtype: float64

📊 ESCALA DE LOS DATOS:
   PEN (niveles) - Mean: 3.1661
   PEN (niveles) - Last: 3.7395

🎯 LO QUE DEBERÍA PASAR:
   Input a Chronos-2: serie de niveles ≈ 3.64
   Output de Chronos-2: siguiente nivel ≈ 3.64
   Conversión a retorno: log(3.64/3.64) ≈ 0.00

❌ LO QUE ESTÁ PASANDO:
   Output actual: 0.000098
   Esto parece un RETORNO, no un NIVEL

🔍 POSIBLES CAUSAS:

1. Se le está pasando RETORNOS en lugar de NIVELES
   → Chronos-2 predice el siguiente "retorno" (pequeño)

2. Se le está pasando NIVELES pero hay una transformación
   interna que convierte a retornos antes de predecir

3. El código calcula retorno ANTES de llamar a Chronos-2
   y le pasa eso como target

Necesito ver el código exacto de cómo se preparan los datos.



In [26]:
print("Config del mejor modelo:")
print(best_results['config'])

Config del mejor modelo:
{'covariates': 0}


In [27]:
# %%
# PRUEBA RÁPIDA: UNIVARIADO CON NIVELES

def rolling_forecast_niveles_univariado(train_df, holdout_df, pipeline):
    """Rolling forecast UNIVARIADO usando NIVELES."""
    
    n_holdout = len(holdout_df)
    pred_returns = []
    
    logger.info(f"\n🔄 Rolling forecast UNIVARIADO (NIVELES)...")
    
    full_df = pd.concat([train_df, holdout_df])
    
    for i in range(n_holdout):
        if i % 10 == 0:
            logger.info(f"   Día {i+1}/{n_holdout}...")
        
        context_end = len(train_df) + i
        context_df = full_df.iloc[:context_end].copy()
        prev_nivel = context_df['PEN'].iloc[-1]
        
        # SOLO niveles de PEN - sin exógenas
        input_df = pd.DataFrame({
            'id': 'USDPEN',
            'timestamp': context_df.index,
            'target': context_df['PEN'].values
        })
        
        # Frecuencia business days
        input_df = input_df.set_index('timestamp').asfreq('B').ffill().reset_index()
        input_df = input_df.rename(columns={'index': 'timestamp'})
        
        try:
            pred_df = pipeline.predict_df(
                input_df, prediction_length=1, quantile_levels=[0.5],
                id_column='id', timestamp_column='timestamp', target='target'
            )
            pred_nivel = float(pred_df['predictions'].iloc[0])
            pred_ret = np.log(pred_nivel / prev_nivel)
            pred_returns.append(pred_ret)
        except Exception as e:
            logger.warning(f"   ⚠️ Error día {i}: {e}")
            pred_returns.append(0.0)
    
    return np.array(pred_returns)

# Ejecutar
predictions = rolling_forecast_niveles_univariado(train_df, holdout_df, pipeline)

# Métricas
y_true = holdout_df['PEN_log_ret'].values
da = (np.sign(y_true) == np.sign(predictions)).mean() * 100
naive_mae = np.mean(np.abs(np.diff(train_df['PEN_log_ret'].values)))
mase = np.mean(np.abs(y_true - predictions)) / naive_mae

logger.info(f"\n{'='*60}")
logger.info(f"📊 RESULTADOS CHRONOS-2 UNIVARIADO (NIVELES)")
logger.info(f"   DA:   {da:.2f}%")
logger.info(f"   MASE: {mase:.4f}")
logger.info(f"{'='*60}")

if da > 50:
    logger.info("✅ ¡Funciona! Ahora podemos probar con exógenas en niveles")
else:
    logger.info("❌ No funciona - Chronos-2 no es útil para USD/PEN")

NameError: name 'pipeline' is not defined

In [13]:
# %%
# ================================================================================
# CELDA 10: GENERAR OOF PARA META-LEARNER
# ================================================================================

logger.info("\n" + "=" * 80)
logger.info("GENERANDO OOF PARA META-LEARNER")
logger.info("=" * 80)

# Usar el mejor experimento para OOF
use_covariates_oof = (best_name == 'Multivariado')

logger.info(f"Usando configuración: {best_name}")
logger.info(f"⏰ Esto puede tomar 60-120 minutos en GPU")

oof_results = forecaster.generate_oof_predictions(
    train_df=train_df,
    target_col=TARGET_COL,
    covariate_cols=EXOG_FEATURES if use_covariates_oof else None,
    use_covariates=use_covariates_oof,
    checkpoint_file=CHECKPOINT_DIR / 'oof_chronos2_checkpoint.pkl'
)

logger.info(f"\n✅ OOF Completado:")
logger.info(f"   Observaciones: {oof_results['n_observations']}")
logger.info(f"   DA OOF: {oof_results['DA']:.2f}%")
logger.info(f"   MASE OOF: {oof_results['MASE']:.4f}")


23:59:32 - INFO - 
23:59:32 - INFO - GENERANDO OOF PARA META-LEARNER
23:59:32 - INFO - ================================================================================
23:59:32 - INFO - Usando configuración: Univariado
23:59:32 - INFO - ⏰ Esto puede tomar 60-120 minutos en GPU
23:59:32 - INFO - 
23:59:32 - INFO - 🔄 GENERANDO OOF (Univariado) - WALK-FORWARD CV
23:59:32 - INFO - ================================================================================
23:59:32 - INFO - 
📊 Configuración:
23:59:32 - INFO -    Observaciones: 8112
23:59:32 - INFO -    Min train: 252
23:59:32 - INFO -    Step size: 21
23:59:32 - INFO -    Splits: 374
23:59:32 - INFO - 
──────────────────────────────────────────────────
23:59:32 - INFO - Fold 1/374 | Context: 0→252 | Val: 252→273
23:59:34 - INFO -    ✅ DA: 52.4% | MAE: 0.002186
23:59:34 - INFO - 
──────────────────────────────────────────────────
23:59:34 - INFO - Fold 2/374 | Context: 0→273 | Val: 273→294
23:59:34 - INFO -    ✅ DA: 47.6% | MAE: 0.00133

In [ ]:
# %%
# ================================================================================
# CELDA 11: EXPORTAR PREDICCIONES (FORMATO ESTÁNDAR)
# ================================================================================

logger.info("\n" + "=" * 80)
logger.info("EXPORTANDO PREDICCIONES")
logger.info("=" * 80)

# ───────────────────────────────────────────────────────────────────────────────
# 1. HOLDOUT PREDICTIONS (para evaluación final)
# ───────────────────────────────────────────────────────────────────────────────

df_holdout_pred = pd.DataFrame({
    'ds': holdout_df.index,
    'y_pred': best_results['predictions'],
    'model': 'Chronos2',
    'type': 'log_returns'
})

holdout_path = PRED_DUMP_DIR / 'pred_Chronos2.csv'
df_holdout_pred.to_csv(holdout_path, index=False)
logger.info(f"✓ Holdout predictions: {holdout_path}")

# ───────────────────────────────────────────────────────────────────────────────
# 2. OOF PREDICTIONS (para meta-learner)
# ───────────────────────────────────────────────────────────────────────────────

df_oof = pd.DataFrame({
    'ds': oof_results['dates'],
    'y_pred': oof_results['predictions'],
    'y_real': oof_results['actuals'],
    'model': 'Chronos2',
    'type': 'log_returns'
})

oof_path = OOF_DIR / 'train_oof_Chronos2.csv'
df_oof.to_csv(oof_path, index=False)
logger.info(f"✓ OOF predictions: {oof_path}")

# Verificar integridad
n_zeros_holdout = (df_holdout_pred['y_pred'] == 0).sum()
n_zeros_oof = (df_oof['y_pred'] == 0).sum()

logger.info(f"\n📊 Verificación:")
logger.info(f"   Holdout: {len(df_holdout_pred)} obs, {n_zeros_holdout} zeros")
logger.info(f"   OOF: {len(df_oof)} obs, {n_zeros_oof} zeros")

if n_zeros_holdout == 0 and n_zeros_oof == 0:
    logger.info("   ✅ Sin zeros - Exportación exitosa")
else:
    logger.warning("   ⚠️ Hay zeros en las predicciones")


In [ ]:
# %%
# ================================================================================
# CELDA 12: GENERAR GRÁFICOS PARA PAPER
# ================================================================================

logger.info("\n" + "=" * 80)
logger.info("GENERANDO GRÁFICOS PARA PAPER")
logger.info("=" * 80)

# Inicializar visualizer
viz = PaperVisualizer(OUTPUT_DIR, FIGURES_DIR, TABLES_DIR)

# ───────────────────────────────────────────────────────────────────────────────
# 1. Predicciones vs Real (mejor experimento)
# ───────────────────────────────────────────────────────────────────────────────

viz.plot_predictions_vs_real(
    dates=holdout_df.index.tolist(),
    y_true=y_holdout,
    y_pred=best_results['predictions'],
    title=f'Chronos-2 ({best_name}): Predictions vs Real - USD/PEN Log Returns',
    filename='chronos2_predictions_vs_real.png'
)

# ───────────────────────────────────────────────────────────────────────────────
# 2. Directional Accuracy
# ───────────────────────────────────────────────────────────────────────────────

viz.plot_directional_accuracy(
    dates=holdout_df.index.tolist(),
    y_true=y_holdout,
    y_pred=best_results['predictions'],
    title=f'Chronos-2 ({best_name}): Directional Accuracy - Holdout Period',
    filename='chronos2_directional_accuracy.png'
)

# ───────────────────────────────────────────────────────────────────────────────
# 3. Ablation Study
# ───────────────────────────────────────────────────────────────────────────────

ablation_for_plot = {
    'Univariado': {'DA': ablation_results['Univariado']['DA'], 
                   'MASE': ablation_results['Univariado']['MASE']},
    'Multivariado\n(13 features)': {'DA': ablation_results['Multivariado']['DA'], 
                                    'MASE': ablation_results['Multivariado']['MASE']}
}

viz.plot_ablation_study(
    ablation_results=ablation_for_plot,
    filename='chronos2_ablation_study.png'
)

# ───────────────────────────────────────────────────────────────────────────────
# 4. Comparación con todos los modelos
# ───────────────────────────────────────────────────────────────────────────────

# Combinar con baselines
all_models_results = BASELINES.copy()
all_models_results['Chronos-2'] = {
    'DA': best_results['DA'],
    'MASE': best_results['MASE']
}

# Ordenar por DA
all_models_sorted = dict(sorted(all_models_results.items(), 
                                 key=lambda x: x[1]['DA'], reverse=True))

viz.plot_model_comparison(
    results=all_models_sorted,
    filename='all_models_comparison.png'
)

logger.info(f"\n✓ Gráficos guardados en: {FIGURES_DIR}")


In [ ]:
# %%
# ================================================================================
# CELDA 13: GENERAR TABLAS PARA PAPER
# ================================================================================

logger.info("\n" + "=" * 80)
logger.info("GENERANDO TABLAS PARA PAPER")
logger.info("=" * 80)

# ───────────────────────────────────────────────────────────────────────────────
# 1. Tabla LaTeX: Comparación de modelos
# ───────────────────────────────────────────────────────────────────────────────

latex_comparison = viz.generate_latex_table(
    results=all_models_sorted,
    caption='Comparison of forecasting models for USD/PEN exchange rate (h=1)',
    label='tab:model_comparison',
    filename='table_model_comparison.tex'
)

# ───────────────────────────────────────────────────────────────────────────────
# 2. Tabla LaTeX: Ablation study Chronos-2
# ───────────────────────────────────────────────────────────────────────────────

ablation_table = {
    'Chronos-2 (Univariate)': ablation_results['Univariado'],
    'Chronos-2 (Multivariate)': ablation_results['Multivariado']
}

latex_ablation = viz.generate_latex_table(
    results=ablation_table,
    caption='Chronos-2 ablation study: Impact of exogenous features on USD/PEN forecasting',
    label='tab:chronos2_ablation',
    filename='table_chronos2_ablation.tex'
)

# ───────────────────────────────────────────────────────────────────────────────
# 3. CSV: Métricas completas
# ───────────────────────────────────────────────────────────────────────────────

# Añadir más métricas
all_models_full = {}
for model, metrics in all_models_sorted.items():
    all_models_full[model] = {
        'DA': metrics['DA'],
        'MASE': metrics['MASE'],
        'MAE': metrics.get('MAE', np.nan),
        'RMSE': metrics.get('RMSE', np.nan)
    }

# Añadir métricas completas de Chronos-2
all_models_full['Chronos-2']['MAE'] = best_results['MAE']
all_models_full['Chronos-2']['RMSE'] = best_results.get('RMSE', np.nan)

df_metrics = viz.generate_metrics_csv(
    results=all_models_full,
    filename='all_models_metrics.csv'
)

logger.info(f"\n✓ Tablas guardadas en: {TABLES_DIR}")

In [ ]:
# %%
# ================================================================================
# CELDA 14: GUARDAR CONFIGURACIÓN Y METADATA
# ================================================================================

logger.info("\n" + "=" * 80)
logger.info("GUARDANDO CONFIGURACIÓN Y METADATA")
logger.info("=" * 80)

# ───────────────────────────────────────────────────────────────────────────────
# 1. Metrics JSON
# ───────────────────────────────────────────────────────────────────────────────

metrics_output = {
    'run_id': datetime.now().strftime('%Y%m%d_%H%M%S'),
    'model': 'Chronos-2',
    'model_id': CHRONOS_MODEL_ID,
    'horizon': f'h={H_FORECAST}',
    
    'configuration': {
        'device': DEVICE,
        'n_holdout': N_HOLDOUT,
        'min_train': MIN_TRAIN,
        'step_size': STEP_SIZE,
        'target': TARGET_COL,
        'n_exog_features': len(EXOG_FEATURES),
        'exog_features': EXOG_FEATURES
    },
    
    'ablation_study': {
        'univariate': {
            'DA': ablation_results['Univariado']['DA'],
            'MASE': ablation_results['Univariado']['MASE']
        },
        'multivariate': {
            'DA': ablation_results['Multivariado']['DA'],
            'MASE': ablation_results['Multivariado']['MASE'],
            'n_covariates': len(EXOG_FEATURES)
        },
        'best_experiment': best_name
    },
    
    'holdout_results': {
        'DA': best_results['DA'],
        'MASE': best_results['MASE'],
        'MAE': best_results['MAE'],
        'n_predictions': len(best_results['predictions']),
        'n_zeros': int(np.sum(best_results['predictions'] == 0))
    },
    
    'oof_results': {
        'DA': oof_results['DA'],
        'MASE': oof_results['MASE'],
        'n_observations': oof_results['n_observations'],
        'n_zeros': oof_results['n_zeros'],
        'elapsed_minutes': round(oof_results['elapsed_minutes'], 2)
    },
    
    'comparison_vs_baselines': {
        model: {
            'DA': metrics['DA'],
            'MASE': metrics['MASE'],
            'delta_DA': round(best_results['DA'] - metrics['DA'], 2),
            'delta_MASE': round(best_results['MASE'] - metrics['MASE'], 4)
        }
        for model, metrics in BASELINES.items()
    },
    
    'data_info': {
        'total_observations': len(df_features),
        'train_observations': len(train_df),
        'holdout_observations': len(holdout_df),
        'train_period': f"{train_df.index[0]} to {train_df.index[-1]}",
        'holdout_period': f"{holdout_df.index[0]} to {holdout_df.index[-1]}"
    },
    
    'timestamp': datetime.now().isoformat()
}

with open(OUTPUT_DIR / 'metrics.json', 'w') as f:
    json.dump(metrics_output, f, indent=2, default=str)

logger.info(f"✓ Metrics: {OUTPUT_DIR / 'metrics.json'}")

# ───────────────────────────────────────────────────────────────────────────────
# 2. Config JSON
# ───────────────────────────────────────────────────────────────────────────────

config_output = {
    'model': 'Chronos-2',
    'model_id': CHRONOS_MODEL_ID,
    'parameters': '~120M',
    'architecture': 'Encoder-only Transformer',
    'training': 'Pre-trained on 100B+ time series observations',
    
    'experiment_config': {
        'target': TARGET_COL,
        'horizon': H_FORECAST,
        'n_holdout': N_HOLDOUT,
        'min_train': MIN_TRAIN,
        'step_size': STEP_SIZE,
        'random_state': RANDOM_STATE
    },
    
    'feature_engineering': {
        'technical_features': ['RSI', 'MACD_line', 'GARCH_vol'],
        'macro_features': ['MXN_log_ret_lag_1', 'CLP_log_ret_lag_1', 
                          'COBRE_log_ret_lag_1', 'MXPE_log_ret_lag_1',
                          'DXY_diff_lag_1', 'UST10Y_diff_lag_1'],
        'temporal_features': ['month_sin', 'month_cos', 
                             'day_of_week_sin', 'day_of_week_cos'],
        'anti_leakage': 'All features use .shift(1) or lagged values'
    },
    
    'best_configuration': {
        'experiment': best_name,
        'use_covariates': best_name == 'Multivariado'
    }
}

with open(OUTPUT_DIR / 'config.json', 'w') as f:
    json.dump(config_output, f, indent=2)

logger.info(f"✓ Config: {OUTPUT_DIR / 'config.json'}")


In [ ]:
# %%
# ================================================================================
# CELDA 15: RESUMEN FINAL
# ================================================================================

logger.info("\n" + "=" * 80)
logger.info("📊 RESUMEN FINAL - CHRONOS-2 USD/PEN FORECASTING")
logger.info("=" * 80)

logger.info(f"\n🏛️ MODELO: Chronos-2 ({CHRONOS_MODEL_ID})")
logger.info(f"   Arquitectura: Encoder-only Transformer (~120M params)")
logger.info(f"   Entrenamiento: Pre-trained en 100B+ observaciones")

logger.info(f"\n📈 ABLATION STUDY:")
logger.info(f"   {'Experimento':<20} {'DA (%)':<10} {'MASE':<10}")
logger.info(f"   {'-'*40}")
logger.info(f"   {'Univariado':<20} {ablation_results['Univariado']['DA']:<10.2f} {ablation_results['Univariado']['MASE']:<10.4f}")
logger.info(f"   {'Multivariado':<20} {ablation_results['Multivariado']['DA']:<10.2f} {ablation_results['Multivariado']['MASE']:<10.4f}")
logger.info(f"   🏆 Mejor: {best_name}")

logger.info(f"\n🎯 RESULTADOS HOLDOUT ({N_HOLDOUT} días):")
logger.info(f"   DA:   {best_results['DA']:.2f}%")
logger.info(f"   MASE: {best_results['MASE']:.4f}")
logger.info(f"   MAE:  {best_results['MAE']:.6f}")

logger.info(f"\n📦 OOF PARA META-LEARNER:")
logger.info(f"   Observaciones: {oof_results['n_observations']}")
logger.info(f"   DA OOF: {oof_results['DA']:.2f}%")
logger.info(f"   MASE OOF: {oof_results['MASE']:.4f}")

logger.info(f"\n📊 COMPARACIÓN VS BASELINES:")
logger.info(f"   {'Modelo':<15} {'DA':<8} {'Δ DA':<10}")
logger.info(f"   {'-'*35}")
for model, metrics in sorted(BASELINES.items(), key=lambda x: x[1]['DA'], reverse=True):
    delta = best_results['DA'] - metrics['DA']
    delta_str = f"+{delta:.2f}" if delta > 0 else f"{delta:.2f}"
    logger.info(f"   {model:<15} {metrics['DA']:<8.2f} {delta_str:<10}")

# Veredicto
if best_results['DA'] > 53.33:  # TimesFM
    verdict = "✅ SUPERIOR - Chronos-2 supera TimesFM (mejor baseline)"
elif best_results['DA'] > 51.67:  # ARX
    verdict = "✅ BUENO - Chronos-2 supera ARX"
elif best_results['DA'] > 50:
    verdict = "⚠️ MARGINAL - Chronos-2 supera Random Walk"
else:
    verdict = "❌ INFERIOR - Chronos-2 no supera Random Walk"

logger.info(f"\n🏅 VEREDICTO: {verdict}")

logger.info(f"\n📁 ARCHIVOS GENERADOS:")
logger.info(f"   Predicciones holdout: {PRED_DUMP_DIR / 'pred_Chronos2.csv'}")
logger.info(f"   Predicciones OOF:     {OOF_DIR / 'train_oof_Chronos2.csv'}")
logger.info(f"   Métricas JSON:        {OUTPUT_DIR / 'metrics.json'}")
logger.info(f"   Config JSON:          {OUTPUT_DIR / 'config.json'}")
logger.info(f"   Figuras:              {FIGURES_DIR}")
logger.info(f"   Tablas LaTeX:         {TABLES_DIR}")

logger.info("\n" + "=" * 80)
logger.info("✅ EXPERIMENTO COMPLETADO")
logger.info("=" * 80)


In [ ]:
# %%
# ================================================================================
# CELDA OPCIONAL: EXPORTAR PREDICCIONES CON VALORES REALES
# ================================================================================

# Para análisis posterior y meta-learner
df_full_export = pd.DataFrame({
    'ds': holdout_df.index,
    'y_real': y_holdout,
    'y_pred_uni': ablation_results['Univariado']['predictions'],
    'y_pred_multi': ablation_results['Multivariado']['predictions'],
    'y_pred_best': best_results['predictions'],
    'model': 'Chronos2',
    'best_config': best_name
})

df_full_export.to_csv(OUTPUT_DIR / 'chronos2_full_predictions.csv', index=False)
logger.info(f"✓ Full predictions: {OUTPUT_DIR / 'chronos2_full_predictions.csv'}")

print("\n" + "="*80)
print("🎉 SCRIPT COMPLETADO EXITOSAMENTE")
print("="*80)
